In [ ]:
# =============================================================================
# CELL 0: Google Drive
# =============================================================================

#The required excel data are also in the online appendix. The google drive paths would need to be replaced or adapted in the respective places

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =============================================================================
# CELL 1: Initialization and Enhanced Memory Module
# =============================================================================

# Install all necessary packages
!pip install -q playwright openai nest_asyncio pandas openpyxl jsonschema langchain langchain-community chromadb sentence-transformers duckduckgo-search beautifulsoup4 requests langgraph

# Install Playwright browser driver
!playwright install --with-deps

# Imports for Enhanced Memory Module
from typing import List, Dict, Any, Optional, Tuple, TypedDict
from datetime import datetime
import json
import uuid
import time
import asyncio
from dataclasses import dataclass
from enum import Enum

# LangChain imports
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from playwright.async_api import async_playwright

# LangGraph imports for Tool Orchestration
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Further imports for tools
import requests
from bs4 import BeautifulSoup
import re

# Tool/Agent Communication Protocol
class MessageType(Enum):
    REQUEST = "request"
    RESPONSE = "response"
    INSIGHT = "insight"
    CHALLENGE = "challenge"
    UPDATE = "update"
    LEARNING_FEEDBACK = "learning_feedback"

@dataclass
class AgentMessage:
    """Structured message for communication"""
    from_agent: str
    to_agent: str
    message_type: MessageType
    content: dict
    priority: int = 1  # 1=low, 2=medium, 3=high
    timestamp: str = None

    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now().isoformat()

@dataclass
class ThoughtNode:
    """Represents a thought in the chain-of-thought process"""
    content: str
    is_consistent: bool
    revision_needed: bool
    timestamp: str
    confidence_score: Optional[float] = None

class AgentState(TypedDict):
    """Shared state between all agents"""
    # Current task
    current_question: str
    current_brand: str
    current_product: Optional[str]
    question_type: str

    # Profile and Memory
    profile: dict
    memory: Any

    # Communication
    messages: List[AgentMessage]
    learning_feedback: Optional[dict]

    # Reasoning and decision
    survey_reasoning: Optional[str]
    bidding_reasoning: Optional[str]
    confidence_score: float
    tool_results: Optional[dict]
    final_answer: int
    final_reasoning: str
    consistency_feedback: Optional[str]
    use_brand_analysis: bool
    needs_challenge: bool
    initial_rating: int
    pre_learning_rating: Optional[int]



class EnhancedMemoryModule:
    """
    Memory Module with:
    - Vector-based Long-Term Memory for semantic retrieval
    - Structured short term memory with context window
    - Episodic memory for important decisions
    - Chain-of-thought history for reasoning tracking
    - Global Memory as Learning Feedback Storage
    """

    def __init__(self, agent_name: str, max_short_term_size: int = 20):
        self.agent_name = agent_name
        self.max_short_term_size = max_short_term_size

        print(f"Initialize memory for {agent_name}...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Long-Term Memory as Vector Store
        unique_id = str(uuid.uuid4())[:8]
        self.long_term_memory = Chroma(
            embedding_function=self.embeddings,
            collection_name=f"agent_{agent_name.replace(' ', '_')}_{unique_id}"
        )

        # Short-Term Memory with limited size
        self.short_term_memory: List[Dict[str, Any]] = []

        # Episodic Memory for important events
        self.episodic_memory: List[Dict[str, Any]] = []

        # Chain-of-Thought History
        self.cot_history: List[ThoughtNode] = []

        # Profile and Reflection
        self.profile_data: Optional[Dict] = None
        self.profile_reflection: Optional[str] = None
        self.actual_self_reflection: Optional[str] = None
        self.ideal_self_reflection: Optional[str] = None

        # Agent - Tool Communication Log
        self.agent_messages: List[AgentMessage] = []

        # Survey ratings tracker for consistency
        self._survey_ratings: Dict[str, Dict[str, int]] = {}

        # Global Memory: Learning Feedback Storage
        self.learning_feedback = {
            "survey_adjustments": {},  # {question_type: {"avg_diff": float, "samples": int}}
            "bidding_adjustments": {},  # {brand: {"avg_diff": float, "samples": int}}
            "profile_based_patterns": {},  # Pattern based on profile similarity
            "accumulated_wisdom": []  # List of insights across multiple agents
        }

        print(f"✅ Memory Module for {agent_name} initialized with Learning Support!")

    def store_profile(self, profile: dict, general_reflection: str, actual_self_reflection: str, ideal_self_reflection: str):
        """Saves profile and its psychological reflections in the LTM"""
        self.profile_data = profile
        self.profile_reflection = general_reflection
        self.actual_self_reflection = actual_self_reflection
        self.ideal_self_reflection = ideal_self_reflection

        # Profile components as separate documents in the Vector Store
        docs = []

        # 1. Demographic data
        demo_text = f"Demographics: {profile['age']} years old {profile['gender']}, works as {profile['occupation']}, income: {profile['income_range']}"
        docs.append(Document(
            page_content=demo_text,
            metadata={"type": "demographics", "timestamp": datetime.now().isoformat()}
        ))

        # 2. Actual Self
        actual_text = f"Actual self traits: {', '.join(profile['actual_traits'])}. Top strength: {profile['actual_top_strength']}"
        docs.append(Document(
            page_content=actual_text,
            metadata={"type": "actual_self", "timestamp": datetime.now().isoformat()}
        ))

        # 3. Ideal Self
        ideal_text = f"Ideal self traits: {', '.join(profile['ideal_traits'])}. Top ideal strength: {profile['ideal_top_strength']}"
        docs.append(Document(
            page_content=ideal_text,
            metadata={"type": "ideal_self", "timestamp": datetime.now().isoformat()}
        ))

        # 4. Add reflections
        docs.extend([
            Document(page_content=general_reflection, metadata={"type": "psychological_reflection", "timestamp": datetime.now().isoformat()}),
            Document(page_content=actual_self_reflection, metadata={"type": "actual_self_reflection", "timestamp": datetime.now().isoformat()}),
            Document(page_content=ideal_self_reflection, metadata={"type": "ideal_self_reflection", "timestamp": datetime.now().isoformat()})
        ])

        # Add all documents to the Vector Store
        self.long_term_memory.add_documents(docs)
        print(f"📝 Profile for {profile['username']} saved in LTM")

    def add_survey_reasoning(self, brand: str, question_code: str, answer: int, reasoning: str, confidence: float = None):
        """Adds survey reasoning to the STM"""
        # Determine the interaction type
        if question_code.startswith("Q9") or question_code.startswith("Q10"):
            interaction_type = "self_congruence"
        elif question_code.startswith("Q11"):
            interaction_type = "attachment"
        else:
            interaction_type = "other"

        # Create Interaction Dictionary
        interaction = {
            "brand": brand,
            "type": interaction_type,
            "question": f"{brand}_{question_code}",
            "answer": answer,
            "reasoning": reasoning,
            "confidence": confidence,
            "timestamp": datetime.now().isoformat()
        }

        # Add to STM
        self.short_term_memory.append(interaction)

        # Limit STM
        if len(self.short_term_memory) > self.max_short_term_size:
            self.short_term_memory = self.short_term_memory[-self.max_short_term_size:]

        # Items indicating high brand attachment (4-5) added in the episodic memory
        if answer >= 4 and interaction_type == "attachment":
            episodic_entry = {
                "brand": brand,
                "type": "high_attachment",
                "reasoning": reasoning,
                "timestamp": datetime.now().isoformat()
            }
            self.episodic_memory.append(episodic_entry)
            print(f"   💾 High Attachment to {brand} - Reasoning saved in Episodic Memory")

        # Update survey ratings tracker
        if brand not in self._survey_ratings:
            self._survey_ratings[brand] = {}
        self._survey_ratings[brand][question_code] = answer

    def add_brand_reasoning(self, brand: str, interaction_type: str, question: str, reasoning: str, rating: int = None):
        """General method for brand reasonings"""
        interaction = {
            "brand": brand,
            "type": interaction_type,
            "question": question,
            "reasoning": reasoning,
            "rating": rating,
            "timestamp": datetime.now().isoformat()
        }

        self.short_term_memory.append(interaction)

        if len(self.short_term_memory) > self.max_short_term_size:
            self.short_term_memory = self.short_term_memory[-self.max_short_term_size:]

    def add_cot_thought(self, thought: str, is_consistent: bool, revision_needed: bool = False, confidence: float = None):
        """Adds a chain-of-thought node"""
        node = ThoughtNode(
            content=thought,
            is_consistent=is_consistent,
            revision_needed=revision_needed,
            timestamp=datetime.now().isoformat(),
            confidence_score=confidence
        )
        self.cot_history.append(node)

    def add_agent_message(self, message: AgentMessage):
        """Saves agent communications with tools"""
        self.agent_messages.append(message)

        # Save important messages in the STM too
        if message.priority >= 2:
            self.short_term_memory.append({
                "type": "agent_communication",
                "from": message.from_agent,
                "message_type": message.message_type.value,
                "content_summary": message.content.get("summary", str(message.content)[:100]),
                "timestamp": message.timestamp
            })

    def add_learning_feedback(self, feedback_message: AgentMessage):
        """Processes and saves learning feedback"""
        if feedback_message.message_type != MessageType.LEARNING_FEEDBACK:
            return

        content = feedback_message.content

        # Survey adjustments
        for question, feedback in content.get("survey_feedback", {}).items():
            if feedback["diff"] is not None:  # Skip -77 values (none values)
                key = question.replace(f"{content.get('brand', '')}_", "")
                if key not in self.learning_feedback["survey_adjustments"]:
                    self.learning_feedback["survey_adjustments"][key] = {
                        "total_diff": 0,
                        "samples": 0,
                        "avg_diff": 0
                    }

                adj = self.learning_feedback["survey_adjustments"][key]
                adj["total_diff"] += feedback["diff"]
                adj["samples"] += 1
                adj["avg_diff"] = adj["total_diff"] / adj["samples"]

        # Bidding adjustments
        for brand, feedback in content.get("bidding_feedback", {}).items():
            if feedback["diff"] is not None:
                if brand not in self.learning_feedback["bidding_adjustments"]:
                    self.learning_feedback["bidding_adjustments"][brand] = {
                        "total_diff": 0,
                        "samples": 0,
                        "avg_diff": 0
                    }

                adj = self.learning_feedback["bidding_adjustments"][brand]
                adj["total_diff"] += feedback["diff"]
                adj["samples"] += 1
                adj["avg_diff"] = adj["total_diff"] / adj["samples"]

        # Store profile-based patterns
        if "profile_match_score" in content:
            pattern = {
                "match_score": content["profile_match_score"],
                "recommendations": content.get("recommendations", {}),
                "timestamp": datetime.now().isoformat()
            }
            self.learning_feedback["profile_based_patterns"][feedback_message.from_agent] = pattern

        # Add to accumulated wisdom
        if "recommendations" in content:
            self.learning_feedback["accumulated_wisdom"].append({
                "insight": content["recommendations"],
                "from_agent": feedback_message.from_agent,
                "timestamp": datetime.now().isoformat()
            })

        print(f"   📚 Learning feedback processed and saved")

    def get_learning_adjustment(self, question_type: str, brand: str = None) -> float:
        """Gets average adaptation for a question type"""
        if question_type in self.learning_feedback["survey_adjustments"]:
            adj = self.learning_feedback["survey_adjustments"][question_type]
            if adj["samples"] > 0:
                return adj["avg_diff"]
        return 0.0

    def get_bidding_adjustment(self, brand: str) -> float:
        """Get average adjustment for bidding a brand"""
        if brand in self.learning_feedback["bidding_adjustments"]:
            adj = self.learning_feedback["bidding_adjustments"][brand]
            if adj["samples"] > 0:
                return adj["avg_diff"]
        return 0.0

    def get_accumulated_wisdom(self) -> List[Dict[str, Any]]:
        """Returns collected wisdom/insights"""
        return self.learning_feedback["accumulated_wisdom"]

    def get_brand_specific_memories(self, brand: str, include_all_brands: bool = False) -> List[Dict[str, Any]]:
        """
        Retrieves memories filtered by brand context.

        Args:
            brand: The current brand to filter for
            include_all_brands: If True, returns all brand memories (for bidding)
                               If False, returns only current brand memories (for survey)

        Returns:
            List of filtered memory entries
        """
        if include_all_brands:
            # For bidding: Return all brand-related memories
            return [m for m in self.short_term_memory
                    if m.get("brand") is not None or m.get("type") == "profile"]
        else:
            # For survey: Return only memories for the current brand
            return [m for m in self.short_term_memory
                    if m.get("brand") == brand or m.get("type") in ["profile", "agent_communication"]]

    def get_brand_reasoning_history(self, brand: str, cross_brand: bool = False) -> List[Dict[str, Any]]:
        """
        Retrieves reasoning history with brand filtering.

        Args:
            brand: The brand to get history for
            cross_brand: If True, includes all brand reasonings (for bidding context)

        Returns:
            List of reasoning entries
        """
        if cross_brand:
            return self.short_term_memory
        else:
            return [m for m in self.short_term_memory if m.get("brand") == brand]

    def get_episodic_memories(self, brand: str = None) -> List[Dict[str, Any]]:
        """Fetches Episodic Memories, optionally filtered by brand"""
        if brand:
            return [m for m in self.episodic_memory if m.get("brand") == brand]
        return self.episodic_memory

    def get_cot_history(self) -> List[ThoughtNode]:
        """Returns the chain-of-thought history"""
        return self.cot_history

    def get_survey_rating(self, brand: str, question_code: str) -> Optional[int]:
        """Obtains a specific survey rating"""
        return self._survey_ratings.get(brand, {}).get(question_code)

    def retrieve_long_term(self) -> Dict[str, Any]:
        """Compatibility method - returns profile and reflections"""
        return {
            "profile_info": self.profile_data,
            "reflective_summary": self.profile_reflection,
            "actual_self_reflection": self.actual_self_reflection,
            "ideal_self_reflection": self.ideal_self_reflection
        }

print("✅ Enhanced Memory Module successfully defined!")

In [ ]:
# =============================================================================
# CELL 2: Tool Orchestrators
# =============================================================================

import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

class BrandPersonalityToolOrchestrator:
    """Tool orchestrator for brand personality research with critical Big Five assessment"""

    def __init__(self):
        self.name = "BrandPersonalityToolOrchestrator"
        # Big Five Personality Model from HuggingFace
        self.big_five_model_name = "Minej/bert-base-personality"
        print("   🧠 Loading Big Five Personality Model...")
        self.tokenizer = BertTokenizer.from_pretrained(self.big_five_model_name)
        self.big_five_model = BertForSequenceClassification.from_pretrained(self.big_five_model_name)
        self.cached_results = {}  # Cache for Brand Personalities

        # Initialize Sentence Transformer for semantic comparisons
        print("   🔍 Loading Sentence Transformer for semantic trait comparisons...")
        self.semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

        print("✅ Brand Personality Tool Orchestrator initialized with critical Big Five assessment")

    def get_brand_personality(self, brand: str, client, model: str) -> Dict[str, Any]:
        """Researches and creates a critical brand personality profile with Big Five"""

        # Check Cache
        if brand in self.cached_results:
            print(f"   📋 Using cached Brand Personality for {brand}")
            return self.cached_results[brand]

        print(f"   🔍 Researching Brand Personality for {brand}...")

        # Critical brand personality description accounting for negative aspects
        system_msg = (
            f"You are a critical brand psychology expert analyzing '{brand}'. "
            "Your analysis must be balanced and realistic, not promotional. "
            "Structure your response as follows:\n\n"
            "1. POSITIVE TRAITS: Key positive characteristics and values (3-4 traits)\n"
            "2. NEGATIVE ASPECTS: Critical weaknesses, limitations, or negative associations (3-4 traits)\n"
            "3. CONTROVERSIAL ELEMENTS: Aspects that divide opinion (2-3 points)\n"
            "4. TARGET DEMOGRAPHIC: Who this brand appeals to and who it alienates\n\n"
            "Be specific and critical. Consider:\n"
            "- Pricing and exclusivity issues\n"
            "- Environmental or ethical concerns\n"
            "- Cultural criticisms\n"
            "- Quality vs. marketing perception gaps\n"
            "- Negative consumer experiences"
        )

        user_msg = f"Provide a critical, balanced personality analysis of {brand}:"

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_msg}
            ],
            temperature=0.4,
            max_tokens=800
        )

        brand_description = resp.choices[0].message.content.strip()
        print(f"   📝 Brand Description generated ({len(brand_description)} characters)")

        # Critical Big Five assessment with penalty for negative aspects
        brand_big_five = self.assess_big_five_critical(brand_description, brand)
        print(f"   🎯 Big Five Scores for {brand}:")
        for trait, score in brand_big_five.items():
            print(f"      - {trait}: {score:.1f}")

        # Extract negative aspects for later use
        negative_aspects = self._extract_negative_aspects(brand_description)

        # Create Brand Profile
        brand_profile = {
            "description": brand_description,
            "big_five": brand_big_five,
            "negative_aspects": negative_aspects,
            "analyzed_at": datetime.now().isoformat()
        }

        # Cache the result
        self.cached_results[brand] = brand_profile
        print(f"   ✅ Brand Personality for {brand} created and cached")

        return brand_profile

    def assess_big_five_critical(self, text: str, brand: str) -> Dict[str, float]:
        """Performs critical Big Five assessment with adjustments"""
        # Standard Big Five Assessment
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = self.big_five_model(**inputs)
            logits = outputs.logits.squeeze().detach().numpy()

        labels = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']

        # Base scores
        scores = {}
        for i, label in enumerate(labels):
            score = 1 / (1 + np.exp(-logits[i])) * 100
            scores[label] = float(score)

        # CRITICAL ADJUSTMENTS
        text_lower = text.lower()

        # Negative keywords
        negative_indicators = {
            'Agreeableness': ['controversial', 'divisive', 'criticism', 'unethical', 'exploitative'],
            'Conscientiousness': ['unreliable', 'quality issues', 'inconsistent', 'shortcuts'],
            'Neuroticism': ['unstable', 'volatile', 'unpredictable', 'crisis'],
            'Openness': ['traditional', 'conservative', 'closed-minded', 'rigid']
        }

        # Apply penalties for negative aspects
        for trait, keywords in negative_indicators.items():
            penalty = sum(5 for keyword in keywords if keyword in text_lower)
            if trait in scores:
                scores[trait] = max(10, scores[trait] - penalty)

        # No brand-specific adjustments
        brand_adjustments = {}

        if brand in brand_adjustments:
            for trait, adjustment in brand_adjustments[brand].items():
                if trait in scores:
                    scores[trait] = max(0, min(100, scores[trait] + adjustment))

        # Round final scores
        return {trait: round(score, 2) for trait, score in scores.items()}

    def _extract_negative_aspects(self, description: str) -> List[str]:
        """Extracts negative aspects from the brand description"""
        negative_aspects = []

        # Search for sections with negative aspects
        lines = description.split('\n')
        capture = False

        for line in lines:
            if any(keyword in line.upper() for keyword in ['NEGATIVE', 'CONTROVERSIAL', 'WEAKNESS', 'LIMITATION']):
                capture = True
            elif any(keyword in line.upper() for keyword in ['POSITIVE', 'TARGET']) and capture:
                capture = False
            elif capture and line.strip():
                negative_aspects.append(line.strip())

        return negative_aspects

    def _calculate_trait_similarity(self, trait1: str, trait2: str) -> float:
        """Calculates semantic similarity between two traits"""
        if not trait1 or not trait2:
            return 0.0

        emb1 = self.semantic_model.encode(str(trait1).lower())
        emb2 = self.semantic_model.encode(str(trait2).lower())

        # Cosine similarity
        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        return float(similarity)

    def _analyze_primary_trait_emphasis(self, primary_trait: str, brand_description: str) -> float:
        """Analyzes how strongly the primary trait is emphasized in the brand description"""
        # Divide description into sentences
        sentences = brand_description.split('.')

        # Search for semantically similar terms to the primary trait
        emphasis_score = 0.0
        trait_embedding = self.semantic_model.encode(primary_trait.lower())

        for sentence in sentences:
            # Extract important words from the sentence
            words = [word.strip().lower() for word in sentence.split() if len(word) > 3]

            for word in words:
                try:
                    word_embedding = self.semantic_model.encode(word)
                    similarity = np.dot(trait_embedding, word_embedding) / (
                        np.linalg.norm(trait_embedding) * np.linalg.norm(word_embedding)
                    )

                    # High similarity indicates emphasis on the trait
                    if similarity > 0.6:  # Threshold for semantic similarity
                        emphasis_score += similarity

                except:
                    continue

        return min(1.0, emphasis_score)  # Normalize to 0-1

    def _analyze_negative_trait_conflicts(self, user_traits: List[str], negative_aspects: List[str]) -> float:
        """Analyzes conflicts between user traits and negative brand aspects"""
        if not user_traits or not negative_aspects:
            return 0.0

        conflict_score = 0.0

        for user_trait in user_traits:
            user_embedding = self.semantic_model.encode(user_trait.lower())

            for negative_aspect in negative_aspects:
                # Extract keywords from negative aspects
                negative_words = [word.strip().lower() for word in negative_aspect.split()
                                if len(word) > 3 and word.lower() not in ['brand', 'company', 'product']]

                for neg_word in negative_words:
                    try:
                        neg_embedding = self.semantic_model.encode(neg_word)

                        # Check for semantic proximity (conflict)
                        similarity = np.dot(user_embedding, neg_embedding) / (
                            np.linalg.norm(user_embedding) * np.linalg.norm(neg_embedding)
                        )

                        # High similarity between user trait and negative aspect = conflict
                        if similarity > 0.5:
                            conflict_score += similarity

                    except:
                        continue

        return min(1.0, conflict_score)  # Normalize to 0-1

    def compare_with_user_reflection(self, brand: str, reflection_type: str, reflection_text: str, client, model: str) -> Dict[str, Any]:
        """Compares Brand Personality with User Reflection using the critical Big Five analysis"""

        # Get Brand Personality (from cache or generate new)
        brand_profile = self.get_brand_personality(brand, client, model)

        print(f"   🤝 Comparing {reflection_type} with {brand} Personality...")

        # Critical Big Five assessment with penalty for negative aspects
        user_big_five = self.assess_user_big_five_critical(reflection_text, reflection_type)
        print(f"   👤 Critical User Big Five Scores ({reflection_type}):")
        for trait, score in user_big_five.items():
            print(f"      - {trait}: {score:.1f}")

        comparison = self.compare_big_five_profiles_critical(user_big_five, brand_profile["big_five"])

        print(f"   📊 Personality Match Score: {comparison['match_score']:.1f}% ({comparison['interpretation']})")
        print(f"   📈 Biggest differences:")
        sorted_diffs = sorted(comparison['differences'].items(), key=lambda x: x[1], reverse=True)
        for trait, diff in sorted_diffs[:3]:
            print(f"      - {trait}: {diff:.1f} Points difference")

        return {
            "brand_personality": brand_profile,
            "user_comparison": {
                "comparison_type": reflection_type,
                "user_big_five": user_big_five,
                "comparison": comparison,
                "reflection_based": True,
                "negative_aspects_considered": len(brand_profile.get("negative_aspects", []))
            }
        }

    def compare_with_user_profile_enhanced(self, brand: str, reflection_type: str, reflection_text: str,
                                         user_profile: dict, client, model: str) -> Dict[str, Any]:
        """Extended comparison method with Primary Trait Analysis and Conflict Detection"""

        # Get Brand Personality
        brand_profile = self.get_brand_personality(brand, client, model)

        print(f"   🤝 Advanced analysis {reflection_type} with {brand} Personality...")

        # Big Five Assessment
        user_big_five = self.assess_user_big_five_critical(reflection_text, reflection_type)

        # Advanced analysis with Primary Traits
        if reflection_type == "actual_self":
            primary_trait = user_profile.get('actual_traits', [''])[0] if user_profile.get('actual_traits') else ''
            user_traits = user_profile.get('actual_traits', [])
        else:  # ideal_self
            primary_trait = user_profile.get('ideal_traits', [''])[0] if user_profile.get('ideal_traits') else ''
            user_traits = user_profile.get('ideal_traits', [])

        # Analyze Primary Trait Emphasis in Brand
        primary_trait_emphasis = 0.0
        if primary_trait:
            primary_trait_emphasis = self._analyze_primary_trait_emphasis(
                primary_trait, brand_profile["description"]
            )
            print(f"   🎯 Primary Trait '{primary_trait}' Emphasis in {brand}: {primary_trait_emphasis:.2f}")

        # Analyze conflicts with negative aspects
        conflict_score = self._analyze_negative_trait_conflicts(
            user_traits, brand_profile.get("negative_aspects", [])
        )
        if conflict_score > 0:
            print(f"   ⚠️ Trait conflict score: {conflict_score:.2f}")

        # Standard comparison
        comparison = self.compare_big_five_profiles_critical(user_big_five, brand_profile["big_five"])

        # Advanced match score calculation
        enhanced_match_score = comparison['match_score']

        # Primary Trait Penalty: When the most important trait is not emphasized in Brand personality
        if primary_trait and primary_trait_emphasis < 0.3:  # Weak emphasis
            primary_trait_penalty = (0.3 - primary_trait_emphasis) * 40  # penalty
            enhanced_match_score = max(0, enhanced_match_score - primary_trait_penalty)
            print(f"   📉 Primary Trait Penalty: -{primary_trait_penalty:.1f} points")

        # Conflict Penalty: Conflicts between user traits and negative brand aspects
        if conflict_score > 0.2:  # Significant conflict
            conflict_penalty = conflict_score * 15  # penalty
            enhanced_match_score = max(0, enhanced_match_score - conflict_penalty)
            print(f"   📉 Conflict Penalty: -{conflict_penalty:.1f} points")

        # Update comparison with enhanced score
        enhanced_comparison = comparison.copy()
        enhanced_comparison['match_score'] = round(enhanced_match_score, 2)
        enhanced_comparison['interpretation'] = self._interpret_match_score_critical(enhanced_match_score)
        enhanced_comparison['primary_trait_emphasis'] = primary_trait_emphasis
        enhanced_comparison['conflict_score'] = conflict_score

        print(f"   📊 Enhanced Match Score: {enhanced_match_score:.1f}% ({enhanced_comparison['interpretation']})")

        return {
            "brand_personality": brand_profile,
            "user_comparison": {
                "comparison_type": reflection_type,
                "user_big_five": user_big_five,
                "comparison": enhanced_comparison,
                "reflection_based": True,
                "primary_trait": primary_trait,
                "primary_trait_emphasis": primary_trait_emphasis,
                "conflict_score": conflict_score,
                "negative_aspects_considered": len(brand_profile.get("negative_aspects", []))
            }
        }

    def assess_user_big_five_critical(self, text: str, reflection_type: str) -> Dict[str, float]:
        """Critical Big Five assessment for user reflections"""
        # Standard Assessment
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = self.big_five_model(**inputs)
            logits = outputs.logits.squeeze().detach().numpy()

        labels = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']

        scores = {}
        for i, label in enumerate(labels):
            score = 1 / (1 + np.exp(-logits[i])) * 100

            # Critical normalization for user scores
            normalized_score = 20 + (score * 0.8)

            scores[label] = float(normalized_score)

        return {trait: round(score, 2) for trait, score in scores.items()}

    def compare_big_five_profiles_critical(self, profile1: Dict[str, float], profile2: Dict[str, float]) -> Dict[str, Any]:
        """Critical comparison of Big Five profiles"""
        differences = {}
        weighted_difference = 0

        # Weights for the big five traits - different relative importance for the relationship between brand and people
        trait_weights = {
            'Agreeableness': 0.8,
            'Conscientiousness': 1.7,
            'Openness': 0.6,
            'Extroversion': 1.9,
            'Neuroticism': 1.1
        }

        for trait in profile1:
            diff = abs(profile1[trait] - profile2[trait])
            differences[trait] = round(diff, 2)

            # Weighted difference
            weight = trait_weights.get(trait, 1.0)
            weighted_difference += diff * weight

        # Average weighted deviation
        total_weight = sum(trait_weights.values())
        avg_weighted_difference = weighted_difference / total_weight

        # Large differences are penalized disproportionately
        match_score = max(0, 100 - (avg_weighted_difference * 1.5) ** 1.2)

        return {
            "differences": differences,
            "average_difference": round(avg_weighted_difference, 2),
            "match_score": round(match_score, 2),
            "interpretation": self._interpret_match_score_critical(match_score)
        }

    def _interpret_match_score_critical(self, score: float) -> str:
        """Interpretation of the match score"""
        if score >= 85:
            return "Good personality match"
        elif score >= 70:
            return "Moderate personality match"
        elif score >= 50:
            return "Weak personality match"
        elif score >= 30:
            return "Poor personality match"
        else:
            return "Very poor personality match"

    def create_challenge_message(self, evaluation: dict, initial_rating: int) -> Optional[AgentMessage]:
        """Creates challenge message"""
        match_score = evaluation.get("user_comparison", {}).get("comparison", {}).get("match_score", 50)
        negative_aspects = evaluation.get("user_comparison", {}).get("negative_aspects_considered", 0)

        # Challenge criteria
        if match_score < 50 and initial_rating >= 4:
            return AgentMessage(
                from_agent=self.name,
                to_agent="MainAgent",
                message_type=MessageType.CHALLENGE,
                content={
                    "summary": "Poor brand congruence detected - rating seems too high",
                    "match_score": match_score,
                    "suggestion": "The brand personality significantly differs from yours. Consider lowering your rating.",
                    "reasoning": f"Critical analysis shows only {match_score:.1f}% match with {negative_aspects} negative aspects",
                    "severity": "high"
                },
                priority=3  # Higher priority for poor matches
            )
        elif match_score < 70 and initial_rating >= 4:
            return AgentMessage(
                from_agent=self.name,
                to_agent="MainAgent",
                message_type=MessageType.CHALLENGE,
                content={
                    "summary": "Weak brand congruence - high rating questionable",
                    "match_score": match_score,
                    "suggestion": "Consider if this moderate alignment justifies such a high rating.",
                    "reasoning": f"Analysis shows {match_score:.1f}% personality match",
                    "severity": "medium"
                },
                priority=2
            )
        elif match_score > 84 and initial_rating <= 2:
            return AgentMessage(
                from_agent=self.name,
                to_agent="MainAgent",
                message_type=MessageType.CHALLENGE,
                content={
                    "summary": "High brand congruence despite low rating",
                    "match_score": match_score,
                    "suggestion": "Your personality aligns well with the brand - the low rating may not reflect this.",
                    "reasoning": f"Analysis shows strong {match_score:.1f}% personality match",
                    "severity": "low"
                },
                priority=2
            )
        return None


# =============================================================================
# ENHANCED FEEDBACK TOOL ORCHESTRATOR WITH SEMANTIC SIMILARITY
# =============================================================================

class FeedbackToolOrchestrator:
    """Enhanced Tool Orchestrator for feedback based on human training data with semantic similarity"""

    def __init__(self, training_data_path: str = None):
        self.name = "FeedbackToolOrchestrator"
        self.training_data_path = training_data_path or '/content/drive/MyDrive/profiles_survey/Training_survey_data.xlsx'

        # Initialize Sentence Transformer for semantic similarity
        print("   🧠 Loading Sentence Transformer for semantic similarity...")
        self.semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

        # Cache for Trait-Embeddings
        self.trait_embeddings_cache = {}

        # One-time loading of training data
        try:
            print(f"   📊 Loading training data from {self.training_data_path}")
            self.training_data = pd.read_excel(self.training_data_path)
            print(f"   ✅ {len(self.training_data)} training examples loaded")
        except Exception as e:
            print(f"⚠️ Warning: Could not load training data: {e}")
            self.training_data = pd.DataFrame()  # Empty DataFrame as Fallback

        # Initialize column_mapping
        self.column_mapping = {
            'nike_actual_consistent': 'Nike_Q9_consistent',
            'nike_actual_mirror': 'Nike_Q9_mirror',
            'nike_ideal_consistent': 'Nike_Q10_consistent',
            'nike_ideal_mirror': 'Nike_Q10_mirror',
            'nike_affection': 'Nike_Q11_Affection',
            'nike_love': 'Nike_Q11_Love',
            'nike_connection': 'Nike_Q11_Connection',
            'nike_passion': 'Nike_Q11_Passion',
            'nike_delight': 'Nike_Q11_Delight',
            'nike_captivation': 'Nike_Q11_Captivation',
            'apple_actual_consistent': 'Apple_Q9_consistent',
            'apple_actual_mirror': 'Apple_Q9_mirror',
            'apple_ideal_consistent': 'Apple_Q10_consistent',
            'apple_ideal_mirror': 'Apple_Q10_mirror',
            'apple_affection': 'Apple_Q11_Affection',
            'apple_love': 'Apple_Q11_Love',
            'apple_connection': 'Apple_Q11_Connection',
            'apple_passion': 'Apple_Q11_Passion',
            'apple_delight': 'Apple_Q11_Delight',
            'apple_captivation': 'Apple_Q11_Captivation',
            'levis_actual_consistent': "Levi's_Q9_consistent",
            'levis_actual_mirror': "Levi's_Q9_mirror",
            'levis_ideal_consistent': "Levi's_Q10_consistent",
            'levis_ideal_mirror': "Levi's_Q10_mirror",
            'levis_affection': "Levi's_Q11_Affection",
            'levis_love': "Levi's_Q11_Love",
            'levis_connection': "Levi's_Q11_Connection",
            'levis_passion': "Levi's_Q11_Passion",
            'levis_delight': "Levi's_Q11_Delight",
            'levis_captivation': "Levi's_Q11_Captivation",
            'bid Nike': 'bid_Nike',
            'bid Apple': 'bid_Apple',
            'bid Levis': "bid_Levi's"
        }


    def load_training_data(self, path: str = None):
        """
        Loads training data and column_mapping
        """
        self.column_mapping = {
            'nike_actual_consistent': 'Nike_Q9_consistent',
            'nike_actual_mirror': 'Nike_Q9_mirror',
            'nike_ideal_consistent': 'Nike_Q10_consistent',
            'nike_ideal_mirror': 'Nike_Q10_mirror',
            'nike_affection': 'Nike_Q11_Affection',
            'nike_love': 'Nike_Q11_Love',
            'nike_connection': 'Nike_Q11_Connection',
            'nike_passion': 'Nike_Q11_Passion',
            'nike_delight': 'Nike_Q11_Delight',
            'nike_captivation': 'Nike_Q11_Captivation',
            'apple_actual_consistent': 'Apple_Q9_consistent',
            'apple_actual_mirror': 'Apple_Q9_mirror',
            'apple_ideal_consistent': 'Apple_Q10_consistent',
            'apple_ideal_mirror': 'Apple_Q10_mirror',
            'apple_affection': 'Apple_Q11_Affection',
            'apple_love': 'Apple_Q11_Love',
            'apple_connection': 'Apple_Q11_Connection',
            'apple_passion': 'Apple_Q11_Passion',
            'apple_delight': 'Apple_Q11_Delight',
            'apple_captivation': 'Apple_Q11_Captivation',
            'levis_actual_consistent': "Levi's_Q9_consistent",
            'levis_actual_mirror': "Levi's_Q9_mirror",
            'levis_ideal_consistent': "Levi's_Q10_consistent",
            'levis_ideal_mirror': "Levi's_Q10_mirror",
            'levis_affection': "Levi's_Q11_Affection",
            'levis_love': "Levi's_Q11_Love",
            'levis_connection': "Levi's_Q11_Connection",
            'levis_passion': "Levi's_Q11_Passion",
            'levis_delight': "Levi's_Q11_Delight",
            'levis_captivation': "Levi's_Q11_Captivation",
            'bid Nike': 'bid_Nike',
            'bid Apple': 'bid_Apple',
            'bid Levis': "bid_Levi's"
        }

    def _get_trait_embedding(self, trait: str) -> np.ndarray:
        """Get or calculate embedding for a trait"""
        if trait not in self.trait_embeddings_cache:
            self.trait_embeddings_cache[trait] = self.semantic_model.encode(trait)
        return self.trait_embeddings_cache[trait]

    def _calculate_trait_similarity(self, trait1: str, trait2: str) -> float:
        """Calculates semantic similarity between two traits"""
        if pd.isna(trait1) or pd.isna(trait2) or str(trait1).strip() == '' or str(trait2).strip() == '':
            return 0.0

        emb1 = self._get_trait_embedding(str(trait1).lower())
        emb2 = self._get_trait_embedding(str(trait2).lower())

        # Cosine similarity
        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        return float(similarity)

    def find_best_human_matches(self, profile: dict, n_matches: int = 3) -> List[Tuple[pd.Series, float]]:
        """Finds the n best human matches based on extended profile with semantic similarity"""
        if self.training_data is None:
            self.load_training_data()

        # Calculate similarity score for already covered human counterparts in training data
        match_scores = []

        for idx, human in self.training_data.iterrows():
            score = 0
            weights = 0

            # DEMOGRAPHIC DATA (weighted)
            # Age
            if 'age' in human and pd.notna(human['age']):
                age_diff = abs(profile['age'] - human['age'])
                age_score = max(0, 100 - age_diff)
                score += age_score * 2
                weights += 1

            # Gender
            if 'gender' in human and pd.notna(human['gender']):
                if profile['gender'].lower() == str(human['gender']).lower():
                    score += 100 * 1
                weights += 1

            # Occupation
            if 'occupation' in human and pd.notna(human['occupation']):
                if profile['occupation'].lower() in str(human['occupation']).lower() or \
                   str(human['occupation']).lower() in profile['occupation'].lower():
                    score += 100 * 1
                weights += 1

            # Income
            if 'income' in human and pd.notna(human['income']):
                if profile['income_range'].lower() in str(human['income']).lower() or \
                   str(human['income']).lower() in profile['income_range'].lower():
                    score += 100 * 2
                weights += 1

            # ACTUAL TRAITS (weighted in descending order, as the first characteristic listed best describes the human role counterparts according to the survey)
            trait_weights = [3.0, 2.5, 2.0, 1.5]  # First traits more important
            for i in range(4):
                if f'actual_{i+1}' in human and i < len(profile.get('actual_traits', [])):
                    similarity = self._calculate_trait_similarity(
                        profile['actual_traits'][i],
                        human[f'actual_{i+1}']
                    )
                    score += similarity * 100 * trait_weights[i]
                    weights += trait_weights[i]

            # IDEAL TRAITS (weighted in descending order, as the first characteristic listed best describes the human role counterparts according to the survey)
            for i in range(4):
                if f'ideal_{i+1}' in human and i < len(profile.get('ideal_traits', [])):
                    similarity = self._calculate_trait_similarity(
                        profile['ideal_traits'][i],
                        human[f'ideal_{i+1}']
                    )
                    score += similarity * 100 * trait_weights[i]
                    weights += trait_weights[i]

            # TOP STRENGTHS
            strength_weight = 3.0
            if 'actual_top_strength' in human and pd.notna(human['actual_top_strength']):
                similarity = self._calculate_trait_similarity(
                    profile.get('actual_top_strength', ''),
                    human['actual_top_strength']
                )
                score += similarity * 100 * strength_weight
                weights += strength_weight

            if 'ideal_top_strength' in human and pd.notna(human['ideal_top_strength']):
                similarity = self._calculate_trait_similarity(
                    profile.get('ideal_top_strength', ''),
                    human['ideal_top_strength']
                )
                score += similarity * 100 * strength_weight
                weights += strength_weight

            final_score = score / weights if weights > 0 else 0
            match_scores.append((idx, final_score))

        # Sort by score and take the best n matches
        match_scores.sort(key=lambda x: x[1], reverse=True)

        # Get the actual number of available matches
        available_matches = min(n_matches, len(match_scores))

        if available_matches == 0:
            print(f"   ⚠️ No human matches available")
            return []

        best_matches = []
        for i in range(available_matches):
            idx, score = match_scores[i]
            best_matches.append((self.training_data.iloc[idx], score))

        print(f"   🎯 {available_matches} best Human Matches found")

        return best_matches

    def calculate_feedback(self, agent_responses: dict, human_data: pd.Series, brand: str) -> dict:
        """Calculates feedback for a brand"""
        feedback = {
            "survey_feedback": {},
            "bidding_feedback": {}
        }

        # Survey Feedback
        for orig_col, mapped_col in self.column_mapping.items():
            if brand in mapped_col:
                human_value = human_data.get(orig_col, -77)

                # Skip -77
                if human_value == -77 or pd.isna(human_value):
                    continue

                # Extract Question Code
                question_code = mapped_col.split('_', 1)[1]

                if question_code in agent_responses:
                    agent_value = agent_responses[question_code]
                    diff = human_value - agent_value

                    feedback["survey_feedback"][mapped_col] = {
                        "human": int(human_value),
                        "agent": int(agent_value),
                        "diff": diff
                    }

        # Bidding Feedback
        bid_col = f"bid_{brand}" if brand != "Levi's" else "bid_Levi's"
        if bid_col in self.column_mapping.values():
            orig_bid_col = next(k for k, v in self.column_mapping.items() if v == bid_col)
            human_bid = human_data.get(orig_bid_col, -77)

            if human_bid != -77 and pd.notna(human_bid) and f"bid_{brand}" in agent_responses:
                agent_bid = agent_responses[f"bid_{brand}"]
                diff = human_bid - agent_bid

                feedback["bidding_feedback"][brand] = {
                    "human": float(human_bid),
                    "agent": float(agent_bid),
                    "diff": diff
                }

        return feedback

    def create_feedback_message(self, agent_responses: dict, profile: dict, brand: str) -> AgentMessage:
        """Creates structured feedback message based on the 3 best matches"""
        # Find the 3 best Human Matches
        best_matches = self.find_best_human_matches(profile, n_matches=3)

        if not best_matches:
            # No matches found
            return AgentMessage(
                from_agent=self.name,
                to_agent="MainAgent",
                message_type=MessageType.LEARNING_FEEDBACK,
                content={
                    "summary": "No human matches found for learning",
                    "profile_match_score": 0.0,
                    "brand": brand,
                    "survey_feedback": {},
                    "bidding_feedback": {},
                    "recommendations": {},
                    "matches_used": 0
                },
                priority=3
            )

        # Calculate average feedback over all matches
        aggregated_feedback = {
            "survey_feedback": {},
            "bidding_feedback": {}
        }

        # Collect feedback from all matches
        all_survey_diffs = {}
        all_bidding_diffs = {}

        for human_match, match_score in best_matches:
            single_feedback = self.calculate_feedback(agent_responses, human_match, brand)

            # Aggregate Survey Feedback
            for key, values in single_feedback["survey_feedback"].items():
                if key not in all_survey_diffs:
                    all_survey_diffs[key] = []
                all_survey_diffs[key].append(values["diff"])

            # Aggregate Bidding Feedback
            for key, values in single_feedback["bidding_feedback"].items():
                if key not in all_bidding_diffs:
                    all_bidding_diffs[key] = []
                all_bidding_diffs[key].append(values["diff"])

        # Calculate averages
        for key, diffs in all_survey_diffs.items():
            avg_diff = sum(diffs) / len(diffs)
            aggregated_feedback["survey_feedback"][key] = {
                "human_avg": agent_responses.get(key.split('_', 1)[1], 0) + avg_diff,
                "agent": agent_responses.get(key.split('_', 1)[1], 0),
                "diff": avg_diff,
                "matches_used": len(diffs)
            }

        for key, diffs in all_bidding_diffs.items():
            avg_diff = sum(diffs) / len(diffs)
            aggregated_feedback["bidding_feedback"][key] = {
                "human_avg": agent_responses.get(f"bid_{key}", 0) + avg_diff,
                "agent": agent_responses.get(f"bid_{key}", 0),
                "diff": avg_diff,
                "matches_used": len(diffs)
            }

        # Analyze patterns and create recommendations
        survey_diffs = [f["diff"] for f in aggregated_feedback["survey_feedback"].values()]
        recommendations = {}

        if survey_diffs:
            avg_survey_diff = sum(survey_diffs) / len(survey_diffs)
            if avg_survey_diff > 0.5:
                recommendations["general_tendency"] = f"Your ratings tend to be {abs(avg_survey_diff):.1f} points too low"
            elif avg_survey_diff < -0.5:
                recommendations["general_tendency"] = f"Your ratings tend to be {abs(avg_survey_diff):.1f} points too high"

            # Specific emotion patterns
            emotion_diffs = {k.split('_')[-1]: v["diff"]
                            for k, v in aggregated_feedback["survey_feedback"].items()
                            if "Q11" in k}
            if emotion_diffs:
                if "Love" in emotion_diffs and emotion_diffs["Love"] > 1:
                    recommendations["emotion_adjustment"] = "Love ratings are too conservative"
                elif "Passion" in emotion_diffs and emotion_diffs["Passion"] > 1:
                    recommendations["emotion_adjustment"] = "Passion ratings need to be higher"

        # Bidding Recommendations
        for brand_name, bid_feedback in aggregated_feedback["bidding_feedback"].items():
            if abs(bid_feedback["diff"]) > 20:
                if bid_feedback["diff"] > 0:
                    recommendations[f"bidding_{brand_name}"] = f"Bids for {brand_name} are €{abs(bid_feedback['diff']):.2f} too low"
                else:
                    recommendations[f"bidding_{brand_name}"] = f"Bids for {brand_name} are €{abs(bid_feedback['diff']):.2f} too high"

        # Calculate average match score
        avg_match_score = sum(score for _, score in best_matches) / len(best_matches)

        return AgentMessage(
            from_agent=self.name,
            to_agent="MainAgent",
            message_type=MessageType.LEARNING_FEEDBACK,
            content={
                "summary": f"Learning feedback based on {len(best_matches)} matches (avg {avg_match_score:.1f}% similarity)",
                "profile_match_score": avg_match_score,
                "brand": brand,
                "survey_feedback": aggregated_feedback["survey_feedback"],
                "bidding_feedback": aggregated_feedback["bidding_feedback"],
                "recommendations": recommendations,
                "matches_used": len(best_matches),
                "human_references": [
                    {
                        "age": match.get('age', 'unknown'),
                        "gender": match.get('gender', 'unknown'),
                        "occupation": match.get('occupation', 'unknown'),
                        "match_score": f"{score:.1f}%"
                    }
                    for match, score in best_matches
                ]
            },
            priority=3  # High priority
        )


# =============================================================================
# PRICE RESEARCH TOOL ORCHESTRATOR
# =============================================================================

class PriceResearchToolOrchestrator:
    """Tool Orchestrator for price research"""


    def __init__(self):
        self.name = "PriceResearchToolOrchestrator"
        self.cached_prices = {}

        # URLs that allow scraping
        self.price_configs = {
            "Apple": {
                "url": "https://www.apple.com/de/shop/buy-airpods/airpods-4",
                "product_name": "Apple AirPods 4",
                "selectors": [
                    '[data-test="mms-product-tile"] .price',
                    '.price .sr-only',
                    '.price-current',
                    '[class*="price"] span'
                ]
            },
            "Nike": {
                "url": "https://www.billiger.de/search?searchstring=nike+pegasus+41+erwachsene",
                "product_name": "Nike Pegasus Running Shoes",
                "selectors": [
                    '.product-price .price',
                    '.price-unit',
                    '.product-box-price .price',
                    '[class*="price"]'
                ]
            },
            "Levi's": {
                "url": "https://www.billiger.de/search?searchstring=levis+501+original+jeans+straight+fit",
                "product_name": "Levi's 501 Original Jeans",
                "selectors": [
                    '.price .sr-only',
                    '.price-sales',
                    '.price-current',
                    '[class*="price"] span'
                ]
            }
        }

        print("✅ Price Research Tool Orchestrator initialized")

    def _extract_prices_from_text(self, text: str, brand: str) -> List[float]:
        """Extracts prices from text with different formats"""
        price_patterns = [
            r'€\s*(\d+(?:,\d+)?(?:\.\d+)?)',  # €99.99 or €99,99
            r'(\d+(?:,\d+)?(?:\.\d+)?)\s*€',  # 99.99€ or 99,99€
            r'(\d+(?:\.\d+)?)\s*EUR',         # 99.99 EUR
            r'ab\s*€?\s*(\d+(?:,\d+)?)',      # From €99
            r'(\d{2,3}(?:,\d+)?)\s*€'         # At least 2-digit prices
        ]

        prices = []
        for pattern in price_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                try:
                    # Convert German format to float
                    price_str = match.replace(',', '.')
                    price_float = float(price_str)

                    # Plausibility checks depending on the product
                    if brand == "Apple" and 50 < price_float < 300:  # AirPods
                        prices.append(price_float)
                    elif brand == "Nike" and 30 < price_float < 250:  # Shoes
                        prices.append(price_float)
                    elif brand == "Levi's" and 20 < price_float < 200:  # Jeans
                        prices.append(price_float)

                except ValueError:
                    continue

        return sorted(list(set(prices)))  # Sorted and without duplicates

    def research_price(self, brand: str) -> Dict:
        """Researches prices for a brand"""
        print(f"\n💰 Researching prices for {brand}...")

        if brand not in self.price_configs:
            return {
                "brand": brand,
                "status": "error",
                "message": "Brand not configured"
            }

        config = self.price_configs[brand]
        url = config["url"]
        product_name = config["product_name"]

        print(f"   🌐 Scraping {url}")
        print(f"   📦 Product: {product_name}")

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'de-DE,de;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
        }

        try:
            response = requests.get(url, headers=headers, timeout=15)
            print(f"   📡 HTTP Status: {response.status_code}")

            if response.status_code != 200:
                raise Exception(f"HTTP {response.status_code}")

            soup = BeautifulSoup(response.text, 'html.parser')
            all_prices = []

            # Search for prices with different selectors
            for selector in config["selectors"]:
                try:
                    price_elements = soup.select(selector)

                    for element in price_elements[:10]:
                        text = element.get_text(strip=True)
                        prices = self._extract_prices_from_text(text, brand)
                        all_prices.extend(prices)

                except Exception:
                    continue

            # Fallback: Search entire text for prices
            if not all_prices:
                print("   🔄 Using fallback search...")
                page_prices = self._extract_prices_from_text(response.text[:10000], brand)
                all_prices.extend(page_prices)

            # Remove duplicates and sort
            unique_prices = sorted(list(set(all_prices)))

            if unique_prices:
                min_price = min(unique_prices)
                avg_price = sum(unique_prices[:5]) / min(5, len(unique_prices))

                return {
                    "brand": brand,
                    "product": product_name,
                    "status": "success",
                    "lowest_price": min_price,
                    "average_price": round(avg_price, 2),
                    "all_prices": unique_prices[:10],
                    "source": url
                }
            else:
                # Fallback
                fallback_prices = {
                    "Apple": 129.0,
                    "Nike": 89.0,
                    "Levi's": 59.0
                }

                return {
                    "brand": brand,
                    "product": product_name,
                    "status": "fallback",
                    "lowest_price": fallback_prices.get(brand, 80.0),
                    "message": "No prices found - using fallback",
                    "source": "fallback"
                }

        except Exception as e:
            print(f"   ⚠️ Error: {str(e)}")

            # Fallback
            fallback_prices = {
                "Apple": 129.0,
                "Nike": 89.0,
                "Levi's": 59.0
            }

            return {
                "brand": brand,
                "product": product_name,
                "status": "error",
                "lowest_price": fallback_prices.get(brand, 80.0),
                "error": str(e),
                "source": "fallback"
            }

    def create_price_message(self, product: str) -> AgentMessage:
        """Creates structured price message with optional cache"""
        # Derive brand from product
        brand = next((b for b in self.price_configs if b.lower() in product.lower()), "Unknown")

        # Check cache
        if brand in self.cached_prices:
            price = self.cached_prices[brand]
            source = "cache"
            status = "cached"
            print(f"   🧠 Price for {brand} from Cache: €{price:.2f}")
        else:
            result = self.research_price(brand)
            price = result.get("lowest_price", 80.0)
            self.cached_prices[brand] = price
            source = result.get("source", "unknown")
            status = result.get("status", "unknown")
            print(f"   📦 New price for {brand} searched: €{price:.2f}")

        return AgentMessage(
            from_agent=self.name,
            to_agent="MainAgent",
            message_type=MessageType.RESPONSE,
            content={
                "summary": f"Reference price for {product}: €{price:.2f}",
                "reference_price": price,
                "product": product,
                "source": source,
                "status": status
            },
            priority=2
        )


# Utility function for Brand Attachment score
def calculate_brand_attachment_score(memory: EnhancedMemoryModule, brand: str) -> float:
    """Calculates Brand Attachment Score from survey responses"""
    attachment_questions = ["Q11_Affection", "Q11_Love", "Q11_Connection",
                          "Q11_Passion", "Q11_Delight", "Q11_Captivation"]

    scores = []
    for q in attachment_questions:
        rating = memory.get_survey_rating(brand, q)
        if rating:
            scores.append(rating)

    if not scores:
        return 0.5  # Neutral default

    # Normalize to 0-1 scale
    avg_score = sum(scores) / len(scores)
    return (avg_score - 1) / 4  # Convert 1-5 to 0-1 scale


print("✅ Helper Tool Orchestrators defined!")

In [ ]:
# =============================================================================
# CELL 3: Setup and Profile-Loading
# =============================================================================

import os, re, json, nest_asyncio, pandas as pd
from openai import OpenAI
from google.colab import drive

nest_asyncio.apply()
drive.mount('/content/drive')

# UniGPT-Client
os.environ["OPENAI_API_KEY"]  = "****"
os.environ["OPENAI_API_BASE"] = "https://gpt.uni-muenster.de/v1"

client = OpenAI(
    api_key  = os.environ["OPENAI_API_KEY"],
    base_url = os.environ["OPENAI_API_BASE"]
)
model = "Llama-3.3-70B"
print(f"✅ LLM Client configured: {model}")

# Import profile data from Excel
print("\n📊 Load profiles from Excel...")
df_profiles = pd.read_excel('/content/drive/MyDrive/profiles_survey/All_profiles.xlsx')
print(f"   Found: {len(df_profiles)} Profile")

# Helper function for filtering empty traits
def filter_traits(row, trait_type, max_count=4):
    """Filters empty/NaN traits from a row"""
    traits = []
    for i in range(1, max_count + 1):
        trait_value = row[f"{trait_type}_{i}"]
        # Check for NaN, None, empty strings or whitespace only
        if pd.notna(trait_value) and str(trait_value).strip():
            traits.append(str(trait_value).strip())
    return traits

# Convert profiles to structured format
profiles = []
for _, row in df_profiles.iterrows():
    actual_traits = filter_traits(row, "actual")
    ideal_traits = filter_traits(row, "ideal")

    profiles.append({
        "age":                  int(row.age),
        "income_range":         row.income,
        "occupation":           row.occupation,
        "gender":               row.gender,
        "actual_traits":        actual_traits,
        "actual_top_strength":  row.actual_top_strength,
        "ideal_traits":         ideal_traits,
        "ideal_top_strength":   row.ideal_top_strength,
        "username":             f"{row.username} Agent"
    })

# Translation function
def translate_to_english(text: str) -> str:
    """Translates text into English (if necessary)"""
    res = client.chat.completions.create(
        model   = model,
        messages= [
            {"role":"system","content":"You are a precise translator. If the input is already in English, return it exactly as-is. Otherwise translate literally without introducing synonyms or rephrasings."},
            {"role":"user","content":f"Translate into English (or return unchanged): \"{text}\""}
        ],
        temperature=0.1
    )
    return res.choices[0].message.content.strip()

# Pre-processing function for profiles
def preprocess_profile(profile: dict) -> dict:
    """Translates all traits of a profile into English"""
    p = dict(profile)
    p["actual_traits"]       = [translate_to_english(t) for t in p["actual_traits"]]
    p["actual_top_strength"] = translate_to_english(p["actual_top_strength"])
    p["ideal_traits"]        = [translate_to_english(t) for t in p["ideal_traits"]]
    p["ideal_top_strength"]  = translate_to_english(p["ideal_top_strength"])
    return p

# Translate all profiles
print("\n🌐 Translate profiles into English...")
profiles_en = []
for i, p in enumerate(profiles):
    print(f"   Process {p['username']}... ", end="")
    profiles_en.append(preprocess_profile(p))
    print("✓")

# Reflection functions
def reflect_profile(profile: dict) -> str:
    """Creates a compact psychological summary of the profiles"""
    system_msg = (
        "You are a psychologist. Create a SINGLE PARAGRAPH psychological summary "
        "that integrates the person's actual self, ideal self, actual top strength, ideal top strength and demographics "
        "into a cohesive personality profile. Focus on the key psychological dynamics "
        "and self-concept. Keep it concise but insightful."
    )

    user_content = (
        f"Profile:\n"
        f"- Age: {profile['age']}, Gender: {profile['gender']}\n"
        f"- Occupation: {profile['occupation']}, Income: {profile['income_range']}\n"
        f"- Actual Self: {', '.join(profile['actual_traits'])} (top: {profile['actual_top_strength']})\n"
        f"- Ideal Self: {', '.join(profile['ideal_traits'])} (top: {profile['ideal_top_strength']})\n"
    )

    resp = client.chat.completions.create(
        model    = model,
        messages = [
            {"role":"system", "content": system_msg},
            {"role":"user",   "content": user_content}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return resp.choices[0].message.content.strip()

def reflect_actual_self(profile: dict) -> str:
    """Creates a psychological expert reflection on the Actual Self"""
    system_msg = (
        "You are a psychology expert. Analyze how this person sees themselves (actual self) "
        "based on their traits and top strength. Consider that traits are listed in descending order of importance. "
        "Focus on their self-perception, self-concept, and how they view their current personality. "
        "Keep it focused and concise."
    )

    user_content = (
        f"Actual Self Analysis:\n"
        f"- Actual traits (in order of importance): {', '.join(profile['actual_traits'])}\n"
        f"- Actual top strength: {profile['actual_top_strength']}\n"
        f"How does this person see themselves? What is their self-concept?"
    )

    resp = client.chat.completions.create(
        model    = model,
        messages = [
            {"role":"system", "content": system_msg},
            {"role":"user",   "content": user_content}
        ],
        temperature=0.7,
        max_tokens=300
    )
    return resp.choices[0].message.content.strip()

def reflect_ideal_self(profile: dict) -> str:
    """Creates a psychological expert reflection on the Ideal Self"""
    system_msg = (
        "You are a psychology expert. Analyze this person's ideal self and aspirations "
        "based on their ideal traits and top ideal strength. Consider that traits are listed in descending order of importance. "
        "Focus on their aspirations, what they want to become, and gaps between actual and ideal self. "
        "Keep it focused and concise."
    )

    user_content = (
        f"Ideal Self Analysis:\n"
        f"- Ideal traits (in order of importance): {', '.join(profile['ideal_traits'])}\n"
        f"- Ideal top strength: {profile['ideal_top_strength']}\n"
        f"What does this person aspire to be? What are their self-improvement goals?"
    )

    resp = client.chat.completions.create(
        model    = model,
        messages = [
            {"role":"system", "content": system_msg},
            {"role":"user",   "content": user_content}
        ],
        temperature=0.7,
        max_tokens=300
    )
    return resp.choices[0].message.content.strip()

# Initialize helper tool orchestrators (global, one-time)
print("\n🤖 Initialize Helper Tool Orchestrators...")
# Renaming
brand_personality_agent = BrandPersonalityToolOrchestrator()
price_research_agent = PriceResearchToolOrchestrator()
feedback_agent = FeedbackToolOrchestrator()

print("\n✅ Setup done!")
print(f"   Number Profiles: {len(profiles_en)}")
print("   Helper Tool Orchestrators: Ready")

brands = ["Nike", "Apple", "Levi's"]

In [ ]:
# =============================================================================
# CELL 4: Agent Enhanced Survey
# =============================================================================

from langgraph.graph import StateGraph, END

# Question templates
actual_qs = {
    "Q9_consistent": "The personality of {brand} is consistent with my actual self.",
    "Q9_mirror":     "The personality of {brand} is a mirror image of my actual self."
}
ideal_qs = {
    "Q10_consistent": "The personality of {brand} is consistent with how I ideally would like to be.",
    "Q10_mirror":     "The personality of {brand} is a mirror image of the person I ideally would like to be."
}

attachment_items = ["Affection", "Love", "Connection", "Passion", "Delight", "Captivation"]

class AgentSurveyOrchestrator:
    """Orchestrates the Agent Survey with LangGraph, Learning and Brand Analysis Tracking"""

    def __init__(self, memory: EnhancedMemoryModule, profile: dict, shared_memory: EnhancedMemoryModule):
        self.memory = memory
        self.profile = profile
        self.shared_memory = shared_memory  # Global Memory
        self.workflow = self._build_workflow()

    def _build_workflow(self) -> StateGraph:
        """Orchestrates tools with LangGraph """
        workflow = StateGraph(AgentState)

        # Define nodes
        workflow.add_node("initial_reasoning", self.initial_reasoning)
        workflow.add_node("confidence_check", self.check_confidence)
        workflow.add_node("brand_analysis", self.brand_personality_analysis)
        workflow.add_node("integrate_challenge", self.integrate_challenge)
        workflow.add_node("finalize_answer", self.finalize_answer)
        workflow.add_node("apply_learning", self.apply_learning_adjustment)

        # Define Edges
        workflow.set_entry_point("initial_reasoning")
        workflow.add_edge("initial_reasoning", "confidence_check")

        # Conditional routing based on Confidence
        workflow.add_conditional_edges(
            "confidence_check",
            self.should_use_brand_analysis,
            {
                True: "brand_analysis",
                False: "finalize_answer"
            }
        )

        # Challenge integration if necessary
        workflow.add_conditional_edges(
            "brand_analysis",
            self.needs_challenge,
            {
                True: "integrate_challenge",
                False: "finalize_answer"
            }
        )

        workflow.add_edge("integrate_challenge", "finalize_answer")
        workflow.add_edge("finalize_answer", "apply_learning")
        workflow.add_edge("apply_learning", END)

        return workflow.compile()

    def _get_few_shot_examples(self, question_type: str) -> str:
        """Returns Few-Shot examples for the respective question type to ensure the correct format"""

        if question_type == "Q9_consistent":
            return """
Example for Q9_consistent (Actual Self):
Statement: "The personality of [Brand] is consistent with my actual self."

Initial reaction: Looking at my actual self - hardworking, practical, down-to-earth - I initially feel some alignment with [Brand]. Starting rating: 4

Arguments FOR agreement:
- [Brand] emphasizes performance and achievement, which resonates with my hardworking nature
- Their achiever mentality aligns with my action-oriented approach
- The brand's athletic focus matches my preference for practical, functional products

Arguments AGAINST agreement:
- As a 45-year-old office worker with modest income, I'm not really their target demographic of young athletes
- The brand's premium pricing conflicts with my practical, value-conscious nature and their association with elite sports and celebrity endorsements feels distant from my everyday reality
- Their biggest competitor adidas aligns better with me
- Recent controversies about labor practices clash with my ethical values

Critical weighing: While there's some surface-level alignment with achievement and action, the deeper analysis reveals significant disconnects. My practical, middle-class reality doesn't match their premium, youth-oriented image. The ethical concerns particularly trouble me.
So I neither agree nor disagree that the personality of [Brand] is consistent with my actual self. Therefore my rating changed to 3.

RATING I: 3
"""

        elif question_type == "Q10_consistent":
            return """
Example for Q10_consistent (Ideal Self):
Statement: "The personality of [Brand] is consistent with how I ideally would like to be."

Initial reaction: My ideal self aspires to be innovative and creative. [Brand] has some innovative aspects. Starting rating: 3

Arguments FOR agreement:
- [Brand] represents innovation in technology, which aligns with my desire to be more tech-savvy
- Their minimalist aesthetic appeals to my ideal of being more organized and focused
- The brand's association with creativity in their marketing resonates with my aspirations

Arguments AGAINST agreement:
- The brand's elitist image conflicts with my ideal of being inclusive and approachable
- Their closed ecosystem goes against my ideal value of openness and flexibility, which is important to me as a young student
- The high price points don't align with my ideal of being financially responsible and my income
- Their corporate culture seems demanding and stressful, not the balanced life I aspire to

Critical weighing: While [Brand]'s innovation is appealing, their exclusivity and closed approach fundamentally conflicts with my ideal values of openness and inclusivity. So I disagree that the personality of [Brand] is consistent with how I ideally would like to be. Therefore my rating changed to 2.

RATING I: 2
"""

        elif question_type.startswith("Q11_Affection"):
            return """
Example for Q11_Affection (Brand Attachment):
Statement: "My feelings toward [Brand] can be characterized by Affection."

Initial reaction: I have mixed feelings about [Brand]. They make decent products but I do not see myself as someone who feels emotions toward a brand. Starting rating: 1

Arguments FOR affection:
- I've owned their jeans for years and they've been reliable companions
- There's some nostalgia from wearing them in my younger days
- I appreciate their durability and classic style
- Based on my memory it seems like that the brand and I do have some similar beliefs

Arguments AGAINST affection:
- It's more habit than affection - I buy them because they're available, not out of love
- No emotional excitement when I see their products or advertising
- Their recent quality seems to have declined while prices increased
- Other brands offer better value and innovation now
- The relationship feels transactional rather than emotional

Critical weighing: While there's some mild positive association from past experiences, true affection requires emotional warmth that's simply not there. So I disagree that my feelings toward the brand can be characterized by affection. Therefore my rating changed to 2.

RATING I: 2
"""

        else:
            # For other Q11 emotions a generic example
            return """
Example for Brand Attachment:
Follow the same structure: Start with initial rating, list FOR/AGAINST arguments considering your demographics and the specific emotion, then critically weigh to reach final rating.
"""

    def initial_reasoning(self, state: AgentState) -> AgentState:
        """Conducts structured pro/con CoT reasoning"""
        question_type = state["question_type"]
        brand = state["current_brand"]

        # Get brand-specific memories (not cross-brand for survey)
        brand_specific_memories = self.memory.get_brand_specific_memories(brand, include_all_brands=False)

        # Get relevant reflection
        ltm_data = self.memory.retrieve_long_term()

        # Determine which reflection to use based on question type
        if question_type.startswith("Q9"):
            context = ltm_data["actual_self_reflection"]
            focus = "actual self"
        elif question_type.startswith("Q10"):
            context = ltm_data["ideal_self_reflection"]
            focus = "ideal self"
        else:  # Q11
            context = ltm_data["reflective_summary"]
            focus = "overall personality"

        # Include demographic data
        demographics = f"""Demographics:
        - Age: {self.profile['age']} years old
        - Gender: {self.profile['gender']}
        - Occupation: {self.profile['occupation']}
        - Income: {self.profile['income_range']}"""

        # Only use brand-specific history
        history_context = ""
        brand_memories = [m for m in brand_specific_memories if m.get("brand") == brand and "reasoning" in m]
        if brand_memories:
            recent = brand_memories[-2:]
            history_context = f"\nPrevious thoughts about {brand}:\n"
            history_context += "\n".join([f"- {h['reasoning'][:600]}..." for h in recent])

        # Structured system prompt with scale definition and few-shot examples
        few_shot_examples = self._get_few_shot_examples(question_type)

        system_msg = (
            f"You are answering a survey SPECIFICALLY about {brand} (not any other brand). "
            f"Think step by step about {brand} based on your {focus} AND your demographics. "
            "IMPORTANT SCALE DEFINITION:\n"
            "1 = Strongly disagree\n"
            "2 = Disagree\n"
            "3 = Neither agree nor disagree\n"
            "4 = Agree\n"
            "5 = Strongly agree\n\n"
            "Follow this structured reasoning process:\n"
            "1. Find 2-3 arguments FOR agreement \n"
            "2. Find 2-3 arguments AGAINST agreement (including any negative associations)\n"
            "3. Consider all facets of the brand - not just surface level\n"
            "4. Start with an initial rating, then adjust it as you consider each argument\n"
            "5. Critically weigh which arguments are stronger\n"
            "6. End with a final rating based on your analysis\n\n"
            f"EXAMPLE REASONING:\n{few_shot_examples}"
        )

        # User Prompt
        user_msg = (
            f"CURRENT BRAND: {brand}\n\n"
            f"My {focus}:\n{context}\n\n"
            f"{demographics}\n\n"
            f"{history_context}\n\n"
            f"Statement about {brand}: {state['current_question']}\n\n"
            "Analyze this statement considering who I am as a person "
            f"and all facets of {brand}. Does this statement make sense for ME specifically?\n\n"
            "Structure your response as follows:\n"
            "1. Initial reaction and starting rating (1-5)\n"
            "2. Arguments FOR agreement (2-3 points)\n"
            "3. Arguments AGAINST agreement (2-3 points, include negative associations)\n"
            "4. Critical weighing of arguments\n"
            "5. Final rating with explanation\n\n"
            "Important: End your response with a clear rating in this format:\n"
            "RATING I: [number 1-5]"
        )

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_msg}
            ],
            temperature=0.7,
            max_tokens=1000
        )

        reasoning = resp.choices[0].message.content.strip()

        # Validate brand consistency
        reasoning = self._validate_brand_consistency(reasoning, brand)

        state["survey_reasoning"] = reasoning

        # Extract initial rating from structured reasoning
        initial_rating = self._extract_rating_from_structured_reasoning(reasoning)

        state["initial_rating"] = initial_rating

        # Calculate Confidence Score
        confidence = self._calculate_confidence(reasoning)
        state["confidence_score"] = confidence

        # Save memories
        self.memory.add_cot_thought(reasoning, True, False, confidence)

        print(f"   🧠 Initial Reasoning - Rating I: {initial_rating} (Confidence: {confidence:.1f}/10)")

        return state

    def _extract_rating_from_structured_reasoning(self, reasoning: str) -> int:
        """Extracts the final rating from the structured reasoning"""
        rating_match = re.search(r'RATING I:\s*(\d)', reasoning, re.IGNORECASE)
        if rating_match:
            rating = int(rating_match.group(1))
            print(f"      ✓ Rating aus 'RATING I' extrahiert: {rating}")
            return rating

        # Fallback: Search for other rating patterns
        patterns = [
            r'final rating is\s*(\d)',
            r'rate this\s*(\d)',
            r'rating:\s*(\d)',
            r'give it a\s*(\d)'
        ]

        for pattern in patterns:
            match = re.search(pattern, reasoning, re.IGNORECASE)
            if match:
                rating = int(match.group(1))
                if 1 <= rating <= 5:
                    print(f"      ⚡ Rating from Fallback-Pattern '{pattern}' extracted: {rating}")
                    return rating

        # Ultimate fallback based on content
        print(f"      ⚠️ WARNING: No explicit rating found - use fallback estimate")
        fallback_rating = self._estimate_rating_from_reasoning(reasoning)
        print(f"      ⚠️ Fallback-Rating estimate: {fallback_rating}")
        return fallback_rating

    def _validate_brand_consistency(self, reasoning: str, current_brand: str) -> str:
        """Validates and corrects brand inconsistencies"""
        other_brands = [b for b in brands if b != current_brand]

        # Check if other brands are mentioned
        for other_brand in other_brands:
            if other_brand.lower() in reasoning.lower():
                print(f"   ⚠️ Brand inconsistency detected: {other_brand} in {current_brand} reasoning")
                # Remove the problematic reasoning and generate new
                return f"Thinking about {current_brand} specifically, {reasoning.split('.')[0]}."

        return reasoning

    def _calculate_confidence(self, reasoning: str) -> float:
        """Calculates Confidence Score from Reasoning"""
        # Uncertainty markers
        uncertainty_markers = [
            "not sure", "uncertain", "maybe", "perhaps", "might",
            "hard to say", "difficult", "unclear", "don't know",
            "conflicted", "mixed", "ambivalent", "tough call"
        ]

        reasoning_lower = reasoning.lower()
        uncertainty_count = sum(1 for marker in uncertainty_markers if marker in reasoning_lower)

        # Base confidence
        confidence = 7.0

        # Reduce for each uncertainty marker
        confidence -= uncertainty_count * 0.8

        # Boost for clear statements
        if any(word in reasoning_lower for word in ["definitely", "certainly", "absolutely", "clearly", "strongly"]):
            confidence += 1.5

        # Check for balanced arguments (indicates thoughtful analysis)
        if "however" in reasoning_lower and "but" in reasoning_lower:
            confidence += 0.5

        return max(1.0, min(10.0, confidence))

    def check_confidence(self, state: AgentState) -> AgentState:
        """Checks confidence and decides on brand analysis"""
        confidence = state["confidence_score"]
        question_type = state["question_type"]

        # Brand Analysis only for Q9/Q10 consistent
        if question_type in ["Q9_consistent", "Q10_consistent"]:
            # Threshold: At medium/low confidence → Brand Analysis
            state["use_brand_analysis"] = confidence < 7.5

            # Always analyze if first interaction
            if not self.memory.get_brand_reasoning_history(state["current_brand"]):
                state["use_brand_analysis"] = True
                print(f"   💡 First {state['current_brand']} Interaction - Brand Analysis activated")
        else:
            state["use_brand_analysis"] = False

        return state

    def should_use_brand_analysis(self, state: AgentState) -> bool:
        """Routing-Function for Brand Analysis"""
        return state.get("use_brand_analysis", False)

    def brand_personality_analysis(self, state: AgentState) -> AgentState:
        """Performs Brand Personality Analysis with Big Five"""
        brand = state["current_brand"]
        question_type = state["question_type"]

        # Determine which reflection to use
        if question_type == "Q9_consistent":
            reflection_type = "actual_self"
            reflection_text = self.memory.actual_self_reflection
        else:  # Q10_consistent
            reflection_type = "ideal_self"
            reflection_text = self.memory.ideal_self_reflection


        evaluation = brand_personality_agent.compare_with_user_profile_enhanced(
            brand,
            reflection_type,
            reflection_text,
            self.profile,
            client,
            model
        )

        state["tool_results"] = evaluation

        # Create challenge if necessary - use initial_rating instead of estimate
        initial_rating = state["initial_rating"]
        challenge_msg = brand_personality_agent.create_challenge_message(evaluation, initial_rating)

        if challenge_msg:
            state["messages"].append(challenge_msg)
            self.memory.add_agent_message(challenge_msg)
            state["needs_challenge"] = True
        else:
            state["needs_challenge"] = False

        return state

    def needs_challenge(self, state: AgentState) -> bool:
        """Routing function for challenge integration"""
        return state.get("needs_challenge", False)

    def integrate_challenge(self, state: AgentState) -> AgentState:
        """Integrates challenge from Brand Personality Tool Orchestrator"""
        challenge = next((msg for msg in state["messages"] if msg.message_type == MessageType.CHALLENGE), None)

        if not challenge:
            return state

        print(f"   ⚡ Challenge received: {challenge.content['summary']}")

        # Brand-specific challenge handling
        brand = state["current_brand"]
        initial_rating = state["initial_rating"]

        system_msg = (
            f"You received critical feedback about your {brand} brand assessment. "
            f"Consider this feedback while staying true to your personality. "
            f"Remember: This is about {brand} only.\n\n"
            "SCALE: 1=Strongly disagree, 2=Disagree, 3=Neither agree nor disagree, 4=Agree, 5=Strongly agree\n\n"
            "IMPORTANT: Based on this feedback, you may:\n"
            "1. Keep your original rating if you still believe it's justified\n"
            "2. Lower your rating if the feedback suggests a lower level of congruence with the brand.\n"
            "3. Increase your rating if the feedback suggests a higher level of congruence with the brand.\n"
            "End with: RATING II: [number 1-5]"
        )

        user_msg = (
            f"My initial thoughts about {brand}:\n{state['survey_reasoning']}\n\n"
            f"Initial rating: {initial_rating}\n\n"
            f"Critical feedback: {challenge.content['summary']}\n"
            f"Match score: {challenge.content['match_score']}%\n"
            f"Suggestion: {challenge.content['suggestion']}\n"
            f"Severity: {challenge.content.get('severity', 'medium')}\n\n"
            f"Reconsidering my assessment of {brand}.\n"
            "End with: RATING II: [number 1-5]"
        )

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_msg}
            ],
            temperature=0.6,
            max_tokens=1500
        )

        revised_reasoning = resp.choices[0].message.content.strip()
        state["survey_reasoning"] = revised_reasoning  # Update reasoning

        # Extract new rating
        new_rating = self._extract_rating_ii(revised_reasoning)

        # Clamp on valid range 1-5
        new_rating = max(1, min(new_rating, 5))

        # Track Brand Analysis Impact
        rating_change = new_rating - initial_rating
        state["brand_analysis_impact"] = {
            "initial_rating": initial_rating,
            "post_analysis_rating": new_rating,
            "rating_change": rating_change,
            "challenge_received": True,
            "challenge_severity": challenge.content.get('severity', 'medium'),
            "match_score": challenge.content.get('match_score', 0)
        }

        if new_rating > initial_rating:
            print(f"   📝 Rating increased: {initial_rating} → {new_rating}")
        elif new_rating < initial_rating:
            print(f"   📝 Rating lowered:  {initial_rating} → {new_rating}")
        else:
            print(f"   📝 Rating retained: {initial_rating}")


        state["initial_rating"] = new_rating

        # Update confidence (slightly increased due to reflection)
        state["confidence_score"] = min(10.0, state["confidence_score"] + 1.0)

        self.memory.add_cot_thought(f"Revised after challenge: {revised_reasoning}", True, True)

        return state

    def _extract_rating_ii(self, reasoning: str) -> int:
        """Extracts Rating II from the revised reasoning"""
        # Search for "RATING II: X" pattern
        rating_match = re.search(r'RATING II:\s*(\d)', reasoning, re.IGNORECASE)
        if rating_match:
            rating = int(rating_match.group(1))
            return rating

        # Fallback to standard extraction
        return self._extract_rating_from_structured_reasoning(reasoning)

    def finalize_answer(self, state: AgentState) -> AgentState:
        """Finalizes the survey response - only adjustment used for brand analysis"""
        question_type = state["question_type"]
        brand = state["current_brand"]
        initial_rating = state["initial_rating"]

        # Check whether Brand Analysis has been used
        brand_analysis_used = state.get("use_brand_analysis", False)

        # Track Brand Analysis Impact even if there was no challenge
        if brand_analysis_used and "brand_analysis_impact" not in state:
            state["brand_analysis_impact"] = {
                "initial_rating": initial_rating,
                "post_analysis_rating": initial_rating,
                "rating_change": 0,
                "challenge_received": False,
                "challenge_severity": "none",
                "match_score": state.get("tool_results", {}).get("user_comparison", {}).get("comparison", {}).get("match_score", 0)
            }
        elif not brand_analysis_used:
            # No Brand Analysis
            state["brand_analysis_impact"] = {
                "initial_rating": initial_rating,
                "post_analysis_rating": initial_rating,
                "rating_change": 0,
                "challenge_received": False,
                "challenge_severity": "not_used",
                "match_score": None
            }

        if not brand_analysis_used:
            # No brand analysis → Take over initial rating directly
            state["pre_learning_rating"] = initial_rating
            state["final_answer"]        = initial_rating
            state["final_reasoning"]     = state["survey_reasoning"]
            print(f"   ✅ No Brand Analysis → Rating II correctly adopted: {initial_rating}")
        else:
            # Brand Analysis was used → Reasoning may already be adjusted
            # Rating may have already been adjusted in integrate_challenge
            state["pre_learning_rating"] = initial_rating
            state["final_answer"]        = initial_rating
            state["final_reasoning"]     = state["survey_reasoning"]
            print(f"   ✅ Brand Analysis used → Rating II: {initial_rating}")

        return state

    def apply_learning_adjustment(self, state: AgentState) -> AgentState:
        """Applies Learning Adjustment as the last step"""
        question_type = state["question_type"]
        brand = state["current_brand"]
        pre_learning_rating = state.get("pre_learning_rating", 3)

        # Get Learning Adjustment
        learning_adjustment = self.shared_memory.get_learning_adjustment(question_type, brand)

        # Improved limit check
        if learning_adjustment != 0:
            # Determine step based on ±0.3 threshold
            if learning_adjustment >= 0.3:
                step = 1
            elif learning_adjustment <= -0.3:
                step = -1
            else:
                step = 0

            # Check whether customization is actually possible
            if step != 0:
                adjusted_rating = max(1, min(5, pre_learning_rating + step))
                print(f"   📚 Learning Adjustment: {pre_learning_rating} → {adjusted_rating} (Δ{learning_adjustment:+.1f})")
                state["final_answer"] = adjusted_rating
            else:
                print(f"   📚 Learning Adjustment too small for change (Δ{learning_adjustment:+.1f})")
                state["final_answer"] = pre_learning_rating
        else:
            # No learning adjustment necessary
            state["final_answer"] = pre_learning_rating

        # Save to memory with final answer
        self.memory.add_survey_reasoning(
            brand,
            question_type,
            state["final_answer"],
            state["survey_reasoning"],
            state["confidence_score"]
        )

        print(f"   ✅ {question_type}: {state['final_answer']} (Confidence: {state['confidence_score']:.1f})")

        return state

    def _estimate_rating_from_reasoning(self, reasoning: str) -> int:
        """Fallback method for rating estimation from reasoning text"""
        print(f"      🔍 Fallback estimation is performed...")
        r = reasoning.lower()

        # Strong positive indicators
        if any(phrase in r for phrase in ["strongly agree", "definitely agree", "absolutely agree"]):
            print(f"      → Strong positive indicators found → Rating: 5")
            return 5

        # Mild positive indicators
        if any(phrase in r for phrase in ["agree", "align"]):
            # Check ob nicht negiert
            if not any(neg in r for neg in ["don't agree", "not agree", "disagree"]):
                print(f"      → Mild positive indicators found → Rating: 4")
                return 4

        # Strong negative indicators
        if any(phrase in r for phrase in ["strongly disagree", "definitely disagree", "not at all"]):
            print(f"      → Strong negative indicators found → Rating: 1")
            return 1

        # Mild negative indicators
        if any(phrase in r for phrase in ["disagree", "not align", "different from", "not consistent"]):
            print(f"      → Mild negative indicators found → Rating: 2")
            return 2

        # Neutral indicators
        if any(phrase in r for phrase in ["neither", "neutral", "mixed", "some aspects"]):
            print(f"      → Neutral indicators found → Rating: 3")
            return 3

        # Default neutral
        print(f"      → No strong indicators - Default neutral → Rating: 3")
        return 3

    async def process_question(self, brand: str, question_code: str, question_text: str) -> Dict[str, Any]:
        """Processes a single survey question"""
        # Initialize State
        initial_state = {
            "current_question": question_text,
            "current_brand": brand,
            "question_type": question_code,
            "profile": self.profile,
            "memory": self.memory,
            "messages": []
        }

        # Execute workflow
        final_state = await self.workflow.ainvoke(initial_state)

        # Save pre-learning rating for later analysis and brand analysis impact
        return {
            "rating": final_state["final_answer"],
            "pre_learning_rating": final_state.get("pre_learning_rating", final_state["final_answer"]),
            "reasoning": final_state["final_reasoning"],
            "confidence": final_state["confidence_score"],
            "brand_analysis_impact": final_state.get("brand_analysis_impact", {})
        }


async def run_agent_survey(profile: dict, brand: str, memory: EnhancedMemoryModule, shared_memory: EnhancedMemoryModule) -> Tuple[Dict[str, Any], Dict[str, Any], List[Dict[str, Any]]]:
    """Conducts survey with agent system, learning and brand analysis tracking"""
    results = {"username": profile["username"], "brand": brand}
    pre_learning_results = {"username": profile["username"], "brand": brand}
    brand_analysis_tracking = []

    # Create Orchestrator with Shared Memory
    orchestrator = AgentSurveyOrchestrator(memory, profile, shared_memory)

    print(f"\n   📋 {brand} Survey with Agent System:")

    # Process all questions
    all_questions = list(actual_qs.keys()) + list(ideal_qs.keys()) + [f"Q11_{item}" for item in attachment_items]

    for question_code in all_questions:
        # Determine question text
        if question_code in actual_qs:
            question_text = actual_qs[question_code].format(brand=brand)
        elif question_code in ideal_qs:
            question_text = ideal_qs[question_code].format(brand=brand)
        else:
            item = question_code.split("_")[1]
            question_text = f"My feelings toward {brand} can be characterized by {item}."

        # Process question
        result = await orchestrator.process_question(brand, question_code, question_text)

        # Save result
        results[question_code] = result["rating"]
        pre_learning_results[question_code] = result["pre_learning_rating"]

        # Track Brand Analysis Impact
        brand_analysis_impact = result.get("brand_analysis_impact", {})
        if brand_analysis_impact:  # Only if Brand Analysis Impact exists
            brand_analysis_tracking.append({
                "username": profile["username"],
                "brand": brand,
                "question": question_code,
                "initial_rating": brand_analysis_impact.get("initial_rating", 0),
                "post_analysis_rating": brand_analysis_impact.get("post_analysis_rating", 0),
                "rating_change": brand_analysis_impact.get("rating_change", 0),
                "challenge_received": brand_analysis_impact.get("challenge_received", False),
                "challenge_severity": brand_analysis_impact.get("challenge_severity", "none"),
                "match_score": brand_analysis_impact.get("match_score", None),
                "timestamp": datetime.now().isoformat()
            })

    return results, pre_learning_results, brand_analysis_tracking

print("✅ Agent Survey System with brand-specific memory retrieval!")

In [ ]:
# =============================================================================
# CELL 5: Agent Enhanced Bidding with CoT and Learning
# =============================================================================

class AgentBiddingOrchestrator:
    """Orchestrates Agent Bidding with CoT Reasoning and Learning"""

    def __init__(self, memory: EnhancedMemoryModule, profile: dict, shared_memory: EnhancedMemoryModule):
        self.memory = memory
        self.profile = profile
        self.shared_memory = shared_memory
        self.workflow = self._build_workflow()

    def _build_workflow(self) -> StateGraph:
        """Builds the Bidding Workflow with Learning"""
        workflow = StateGraph(AgentState)

        # Nodes
        workflow.add_node("get_price_info", self.get_price_information)
        workflow.add_node("check_learning", self.check_bidding_learning)
        workflow.add_node("initial_cot", self.initial_chain_of_thought)
        workflow.add_node("calculate_bid", self.calculate_final_bid)

        # Flow
        workflow.set_entry_point("get_price_info")
        workflow.add_edge("get_price_info", "check_learning")
        workflow.add_edge("check_learning", "initial_cot")
        workflow.add_edge("initial_cot", "calculate_bid")
        workflow.add_edge("calculate_bid", END)

        return workflow.compile()

    def get_price_information(self, state: AgentState) -> AgentState:
        """Gets price information from Price Research Tool Orchestrator"""
        product = state["current_product"]

        # Create Price Message
        price_msg = price_research_agent.create_price_message(product)

        # Save Message
        self.memory.add_agent_message(price_msg)
        state["messages"] = [price_msg]

        # Extract price info
        state["tool_results"] = {
            "reference_price": price_msg.content["reference_price"]
        }

        print(f"   💰 Reference price: €{price_msg.content['reference_price']:.2f}")

        return state

    def check_bidding_learning(self, state: AgentState) -> AgentState:
        """Checks Learning Feedback for Bidding"""
        product = state["current_product"]

        # Extract Brand from Product
        brand = None
        for b in brands:
            if b.lower() in product.lower():
                brand = b
                break

        if brand:
            # Get Bidding Adjustment
            bidding_adjustment = self.shared_memory.get_bidding_adjustment(brand)

            # Get accumulated wisdom for Bidding
            wisdom = self.shared_memory.get_accumulated_wisdom()
            bidding_wisdom = [w for w in wisdom if f"bidding_{brand}" in str(w.get("insight", {}))]

            learning_context = {
                "brand": brand,
                "adjustment": bidding_adjustment,
                "samples": len(bidding_wisdom),
                "wisdom": bidding_wisdom[-2:] if bidding_wisdom else []
            }

            state["learning_feedback"] = learning_context

            if bidding_adjustment != 0:
                print(f"   📚 Learning Adjustment for {brand} Bidding: €{bidding_adjustment:+.2f}")
        else:
            state["learning_feedback"] = {}

        return state

    def initial_chain_of_thought(self, state: AgentState) -> AgentState:
        """Performs initial Chain-of-Thought Reasoning for Bidding with Learning and cross-brand memories"""
        product = state["current_product"]
        product_desc = state.get("product_description", "")
        reference_price = state["tool_results"]["reference_price"]
        learning_feedback = state.get("learning_feedback", {})

        # Extract Brand
        brand = learning_feedback.get("brand", None)
        if not brand:
            for b in brands:
                if b.lower() in product.lower():
                    brand = b
                    break

        # Get relevant psychological reflections from LTM
        ltm_data = self.memory.retrieve_long_term()
        general_reflection = ltm_data.get("reflective_summary", "")

        # Get ALL brand memories for bidding (cross-brand experiences)
        all_brand_memories = self.memory.get_brand_specific_memories(brand, include_all_brands=True)

        # Create memory context from ALL brand experiences
        memory_context = ""
        if all_brand_memories:
            memory_context = "My brand experiences from the survey:\n"

            # Group by brand for clarity
            brand_groups = {}
            for mem in all_brand_memories:
                mem_brand = mem.get("brand", "Unknown")
                if mem_brand not in brand_groups:
                    brand_groups[mem_brand] = []
                brand_groups[mem_brand].append(mem)

            # Add memories from all brands
            for mem_brand, memories in brand_groups.items():
                if memories:
                    memory_context += f"\n{mem_brand}:\n"
                    # Take most recent/relevant memories
                    relevant_memories = [m for m in memories if "reasoning" in m][-3:]
                    for mem in relevant_memories:
                        q_type = mem.get("question", "").split("_", 2)[-1]
                        memory_context += f"- {q_type}: {mem['reasoning'][:200]}...\n"

        # Learning Context for Bidding
        learning_context = ""
        if learning_feedback.get("adjustment", 0) != 0:
            learning_context = f"\nLearning insight: Previous agents' bids for {brand} tend to be €{learning_feedback['adjustment']:+.2f} different than initial instinct."

        if learning_feedback.get("wisdom"):
            for w in learning_feedback["wisdom"]:
                if "recommendations" in w.get("insight", {}):
                    rec = w["insight"]["recommendations"]
                    if f"bidding_{brand}" in rec:
                        learning_context += f"\n{rec[f'bidding_{brand}']}"

        # Chain-of-Thought for Bidding
        cot_system = (
            "You are making a purchasing decision. Think through this step by step:\n"
            "1. Consider your personality and profile\n"
            "2. Reflect on your relationship with ALL brands from the survey (not just the current product's brand)\n"
            "3. Compare how you feel about different brands\n"
            "4. Assess this specific brand's value to you personally\n"
            "5. Consider the reference price as one factor among many\n"
            "6. Determine what you're willing to pay based strongly on YOUR values and preferences"
        )

        cot_user = (
            f"My self-concept:\n{general_reflection}\n\n"
            f"{memory_context}\n"
            f"{learning_context}\n\n"
            f"Product to bid on:\n"
            f"- Name: {product}\n"
            f"- Description: {product_desc}\n"
            f"- Market reference: Products like this typically cost around €{reference_price:.2f}\n\n"
            "Let me think through my bidding decision step by step, considering all my brand experiences..."
        )

        # Generate Chain-of-Thought
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": cot_system},
                {"role": "user", "content": cot_user}
            ],
            temperature=0.7,
            max_tokens=700
        )

        cot_process = resp.choices[0].message.content.strip()
        state["bidding_reasoning"] = cot_process

        # Consistency Check for Bidding
        consistency_system = (
            "Evaluate if this bidding thought process is consistent with the person's "
            "overall self-concept, profile, and ALL their survey responses across different brands. "
            "Consider if the reasoning makes sense given their traits and previous responses to all brands."
        )

        consistency_user = (
            f"Person's profile:\n"
            f"- Overall self-concept: {general_reflection[:200]}...\n"
            f"- Age: {self.profile['age']}\n"
            f"- Gender: {self.profile['gender']}\n"
            f"- Occupation: {self.profile['occupation']}\n"
            f"- Income: {self.profile['income_range']}\n\n"
            f"Bidding thought process:\n{cot_process}\n\n"
            "Is this bidding reasoning consistent with their profile and all brand experiences? Provide brief feedback."
        )

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": consistency_system},
                {"role": "user", "content": consistency_user}
            ],
            temperature=0.3,
            max_tokens=150
        )

        consistency_feedback = resp.choices[0].message.content.strip()
        state["consistency_feedback"] = consistency_feedback

        # Add CoT to Memory
        self.memory.add_cot_thought(cot_process, True, False)

        print(f"   🧠 Chain-of-Thought Reasoning completed (using cross-brand experiences)")

        return state

    def calculate_final_bid(self, state: AgentState) -> AgentState:
        """Calculates final bid based on CoT with Learning Adjustments"""
        cot_process = state["bidding_reasoning"]
        consistency_feedback = state.get("consistency_feedback", "")
        reference_price = state["tool_results"]["reference_price"]
        learning_feedback = state.get("learning_feedback", {})

        # Final Bid Decision
        bid_system = (
            "Based on the chain of thought and consistency feedback, determine the final bid amount. "
            "The bid should reflect the person's authentic willingness to pay based on their "
            "overall self-concept, values, and relationship with ALL brands from their survey experiences. "
            "Let the person's reasoning guide the decision naturally. "
            "End with ONLY the bid amount as a number on the last line."
        )

        bid_user = (
            f"Chain of thought:\n{cot_process}\n\n"
            f"Consistency feedback:\n{consistency_feedback}\n\n"
            f"Reference price: €{reference_price:.2f}\n\n"
            "Considering everything, my final bid amount in euros:"
        )

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": bid_system},
                {"role": "user", "content": bid_user}
            ],
            temperature=0.5,
            max_tokens=500
        )

        bid_text = resp.choices[0].message.content.strip()

        # Parse Bid
        bid = 0.0
        all_numbers = re.findall(r'€?\s*(\d+(?:\.\d+)?)', bid_text)
        if all_numbers:
            bid = float(all_numbers[-1])

        # Fallback if no number found
        if bid == 0.0:
            bid = reference_price * 0.9  # 90% of reference price as fallback

        # Apply Learning Adjustment
        if learning_feedback.get("adjustment", 0) != 0:
            original_bid = bid
            bid = bid + learning_feedback["adjustment"]
            bid = max(40.0, bid)  # Minimum €40
            if bid != original_bid:
                print(f"      📚 Learning Adjustment applied: €{original_bid:.2f} → €{bid:.2f}")

        # Minimum bid €40
        bid = max(40.0, bid)

        state["final_answer"] = round(bid, 2)
        state["final_reasoning"] = cot_process

        # Extract Brand for storage
        product = state["current_product"]
        brand = learning_feedback.get("brand", "Unknown")
        if brand == "Unknown":
            brand = next((b for b in brands if b.lower() in product.lower()), "Unknown")

        # Save in Memory
        self.memory.add_brand_reasoning(
            brand=brand,
            interaction_type="bidding",
            question=f"WTP for {product}",
            reasoning=f"{cot_process[:200]}... Reference: €{reference_price:.2f}",
            rating=int(bid)
        )

        print(f"   ✅ Final bid: €{bid:.2f}")

        return state

    async def process_bidding(self, product_name: str, product_desc: str) -> Tuple[float, str]:
        """Processes Bidding for a product"""
        # Initialize State
        initial_state = {
            "current_product": product_name,
            "product_description": product_desc,
            "profile": self.profile,
            "memory": self.memory,
            "messages": []
        }

        # Execute Workflow
        final_state = await self.workflow.ainvoke(initial_state)

        return final_state["final_answer"], final_state["final_reasoning"]


async def calculate_agent_bid(
    profile: dict,
    product_name: str,
    product_desc: str,
    memory: EnhancedMemoryModule,
    shared_memory: EnhancedMemoryModule
) -> Tuple[float, str]:
    """Wrapper for Agent Bidding with CoT and Learning"""
    orchestrator = AgentBiddingOrchestrator(memory, profile, shared_memory)
    return await orchestrator.process_bidding(product_name, product_desc)

print("✅ Agent Bidding System!")

In [ ]:
# =============================================================================
# CELL 6: Enhanced Bidding Agent
# =============================================================================

async def run_agent_shop_bot(profile: dict, memory: EnhancedMemoryModule, shared_memory: EnhancedMemoryModule):
    """Runs Bidding Agent"""
    bidding_results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-setuid-sandbox']
        )
        page = await browser.new_page()
        page.set_default_navigation_timeout(60000)
        page.set_default_timeout(60000)

        # Accept username dialog
        page.on("dialog", lambda dialog: asyncio.create_task(
            dialog.accept(profile["username"])
        ))

        print(f"\n🛒 Start Agent Bidding for {profile['username']}")
        print("   Navigate to the website...")

        await page.goto(
            "https://auction-shop-agent.web.app/",
            wait_until="load",
            timeout=60000
        )


        for product_idx in range(3):
            print(f"\n   📦 Product {product_idx+1}/3:")

            await page.wait_for_selector(".product-content h2", timeout=60000)
            product_name = await page.text_content(".product-content h2")
            product_desc = await page.text_content(".product-content p")
            print(f"      Name: {product_name}")

            # Calculate bid with Agent System
            bid, reasoning = await calculate_agent_bid(
                profile, product_name, product_desc, memory, shared_memory
            )

            # Determine brand
            brand = next((b for b in brands if b.lower() in product_name.lower()), "Unknown")

            # Save results
            bidding_results.append({
                "username": profile["username"],
                "product_idx": product_idx + 1,
                "product_name": product_name,
                "product_desc": product_desc,
                "bid": bid,
                "brand": brand,
                "reasoning": reasoning,
                "timestamp": datetime.now().isoformat()
            })

            # Place a bid
            await page.fill("input[type=number]", f"{bid:.2f}")
            await page.click("button")

            # Waiting for next product or final page
            if product_idx < 2:
                await page.wait_for_function(
                    "([selector, oldText]) => document.querySelector(selector)?.textContent !== oldText",
                    arg=[".product-content h2", product_name],
                    timeout=60000
                )
            else:
                await page.wait_for_selector("h1:has-text('Thank you')", timeout=60000)
                print(f"\n   Shopping completed for {profile['username']}!")

        await browser.close()
    return bidding_results

print("✅ Agent shopping defined!")

In [ ]:
# =============================================================================
# CELL 7: Complete Agent Survey Execution with Feedback Loop
# =============================================================================

async def run_complete_agent_survey_for_single_agent(profile: dict, memory: EnhancedMemoryModule, shared_memory: EnhancedMemoryModule):
    """Carries out the complete survey for a single agent and all brands"""

    agent_name = profile["username"]
    survey_results = []
    pre_learning_results = []
    reasoning_data = []
    agent_messages = []

    # Dictionary for agent responses (for feedback)
    agent_responses = {}

    print(f"\n{'='*70}")
    print(f"👤 Agent: {agent_name}")
    print(f"   Actual: {' > '.join(profile['actual_traits'][:2])}...")
    print(f"   Ideal: {' > '.join(profile['ideal_traits'][:2])}...")
    print(f"{'='*70}")

    # Empty STM, CoT History and Messages before each new agent
    memory.short_term_memory.clear()
    memory.cot_history.clear()
    memory.agent_messages.clear()
    memory._survey_ratings.clear()

    # Survey for every brand
    for brand in brands:
        print(f"\n   📋 {brand} Survey with Agent Orchestration:")

        try:
            results, pre_learning = await run_agent_survey(profile, brand, memory, shared_memory)

            # Collect data for DataFrame
            for question_code in ["Q9_consistent", "Q9_mirror", "Q10_consistent", "Q10_mirror"] + [f"Q11_{item}" for item in attachment_items]:
                rating = results[question_code]
                pre_learning_rating = pre_learning[question_code]

                # Save for feedback
                agent_responses[question_code] = rating

                survey_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Save Pre-Learning Results
                pre_learning_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": pre_learning_rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Find the corresponding reasoning from the memory
                reasoning_entry = next(
                    (m for m in memory.short_term_memory
                     if m.get("question", "").endswith(question_code) and m.get("brand") == brand),
                    None
                )

                if reasoning_entry:
                    reasoning_data.append({
                        "username": agent_name,
                        "brand": brand,
                        "question": question_code,
                        "rating": rating,
                        "pre_learning_rating": pre_learning_rating,
                        "reasoning": reasoning_entry["reasoning"],
                        "confidence": reasoning_entry.get("confidence", None)
                    })

            # Collect agent messages for this brand
            brand_messages = [msg for msg in memory.agent_messages
                            if any(brand in str(msg.content) for brand in [brand])]

            for msg in brand_messages:
                agent_messages.append({
                    "username": agent_name,
                    "brand": brand,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

        except Exception as e:
            print(f"   ❌ Error at {brand}: {str(e)}")

    return survey_results, pre_learning_results, reasoning_data, agent_messages, agent_responses

print("✅ Agent Survey Execution System defined!")

In [ ]:
# =============================================================================
# CELL 8: Complete Agent Study
# =============================================================================

async def run_complete_agent_experiment():
    """Runs the complete study"""

    print("🚀 Start study")
    print("="*70)

    # Initialization of global shared memory for learning
    print("\n🧠 Create global shared memory for collective learning...")
    shared_learning_memory = EnhancedMemoryModule(
        agent_name="SharedLearning",
        max_short_term_size=100
    )
    print("✅ Shared Learning Memory")

    # Collection storage for all results
    all_survey_results = []
    all_pre_learning_results = []
    all_reasoning_data = []
    all_survey_messages = []
    all_bidding_results = []
    all_bidding_messages = []
    all_feedback_messages = []
    all_brand_analysis_tracking = []

    total_agents = len(profiles_en)
    start_time = time.time()
    save_interval = 10  # Save all 10 agents

    # Output directories
    output_dir_survey = "/content/drive/MyDrive/agent_survey_results/"
    output_dir_bidding = "/content/drive/MyDrive/agent_bidding_results/"
    output_dir_learning = "/content/drive/MyDrive/agent_learning_results/"

    import os
    os.makedirs(output_dir_survey, exist_ok=True)
    os.makedirs(output_dir_bidding, exist_ok=True)
    os.makedirs(output_dir_learning, exist_ok=True)

    # Base timestamp for consistent file names
    base_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # MAIN LOOP: Complete sequential processing for each agent
    for agent_idx, profile in enumerate(profiles_en):
        agent_name = profile["username"]

        print(f"\n{'#'*80}")
        print(f"# AGENT {agent_idx + 1}/{total_agents}: {agent_name}")
        print(f"{'#'*80}")

        # STEP 1: Profile processing and reflections
        print(f"\n🔮 PHASE 1: PROFILE PROCESSING for {agent_name}")
        print("-" * 50)

        # Create memory module for this agent
        print(f"   🧠 Create memory module for {agent_name}...")
        memory = EnhancedMemoryModule(
            agent_name=agent_name,
            max_short_term_size=20
        )

        # Use global embedding model for efficiency
        if 'embeddings_model' not in globals():
            print("   📥 Load Embedding-Modell...")
            from langchain_community.embeddings import HuggingFaceEmbeddings
            embeddings_model = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
        memory.embeddings = embeddings_model

        # Create psychological reflections
        print("   🔮 Create psychological summaries...")
        general_summary = reflect_profile(profile)
        actual_self_summary = reflect_actual_self(profile)
        ideal_self_summary = reflect_ideal_self(profile)

        # Save profile and all reflections
        memory.store_profile(profile, general_summary, actual_self_summary, ideal_self_summary)

        print(f"   📝 General: {general_summary[:100]}...")
        print(f"   🎯 Actual Self: {actual_self_summary[:80]}...")
        print(f"   ⭐ Ideal Self: {ideal_self_summary[:80]}...")
        print(f"   ✅ Memory Module for {agent_name} created!")

        # Dictionary for all agent responses
        all_agent_responses = {}

        # STEP 2: Survey for this agent
        print(f"\n📋 PHASE 2: SURVEY for {agent_name}")
        print("-" * 50)

        # Updated Call with Brand Analysis Tracking
        agent_survey_results, agent_pre_learning_results, agent_reasoning_data, agent_survey_messages, agent_survey_responses, agent_brand_analysis_tracking = \
            await run_complete_agent_survey_for_single_agent(profile, memory, shared_learning_memory)

        # Collect Survey-results
        all_survey_results.extend(agent_survey_results)
        all_pre_learning_results.extend(agent_pre_learning_results)
        all_reasoning_data.extend(agent_reasoning_data)
        all_survey_messages.extend(agent_survey_messages)
        all_brand_analysis_tracking.extend(agent_brand_analysis_tracking)

        # Update agent responses dictionary
        all_agent_responses.update(agent_survey_responses)

        print(f"\n✅ Survey for {agent_name} done!")

        # Show Brand Attachment Scores
        print("   📊 Brand Attachment Scores:")
        for brand in brands:
            attachment = calculate_brand_attachment_score(memory, brand)
            print(f"      - {brand}: {attachment:.2%}")

        # STEP 3: Bidding for this agent
        print(f"\n🛒 PHASE 3: Bidding for {agent_name}")
        print("-" * 50)

        # Clear agent messages for bidding
        memory.agent_messages.clear()

        try:
            bidding_results = await run_agent_shop_bot(profile, memory, shared_learning_memory)
            all_bidding_results.extend(bidding_results)

            # Add bidding results to agent_responses
            for bid_result in bidding_results:
                brand = bid_result["brand"]
                all_agent_responses[f"bid_{brand}"] = bid_result["bid"]

            # Collect Agent Messages from bidding
            for msg in memory.agent_messages:
                all_bidding_messages.append({
                    "username": agent_name,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

            print(f"\n✅ Bidding for {agent_name} done!")

        except Exception as e:
            print(f"\n   ❌ Critical error when bidding for {agent_name}: {e!s}")

        # SCHRITT 4: FEEDBACK LOOP
        print(f"\n📊 PHASE 4: FEEDBACK for {agent_name}")
        print("-" * 50)

        # Generate feedback for each brand
        for brand in brands:
            feedback_msg = feedback_agent.create_feedback_message(all_agent_responses, profile, brand)

            # Save feedback in shared memory
            shared_learning_memory.add_learning_feedback(feedback_msg)

            # Also save in the individual memory
            memory.add_agent_message(feedback_msg)

            # Collect for logging
            all_feedback_messages.append({
                "username": agent_name,
                "brand": brand,
                "from_agent": feedback_msg.from_agent,
                "to_agent": feedback_msg.to_agent,
                "message_type": feedback_msg.message_type.value,
                "priority": feedback_msg.priority,
                "profile_match_score": feedback_msg.content.get("profile_match_score", 0),
                "matches_used": feedback_msg.content.get("matches_used", 0),
                "recommendations": json.dumps(feedback_msg.content.get("recommendations", {})),
                "timestamp": feedback_msg.timestamp
            })

            print(f"   📚 Feedback generated for {brand} and saved in the shared memory")

        # Show Learning Summary
        print(f"\n   📈 Learning Summary after {agent_name}:")
        print(f"      Survey Adjustments: {len(shared_learning_memory.learning_feedback['survey_adjustments'])} Pattern")
        print(f"      Bidding Adjustments: {len(shared_learning_memory.learning_feedback['bidding_adjustments'])} Pattern")
        print(f"      Accumulated Wisdom: {len(shared_learning_memory.learning_feedback['accumulated_wisdom'])} Insights")

        # Progress bar
        elapsed = time.time() - start_time
        progress = ((agent_idx + 1) / total_agents) * 100
        eta = (elapsed / (agent_idx + 1)) * (total_agents - agent_idx - 1) if agent_idx < total_agents - 1 else 0

        print(f"\n📈 Overall progress: {progress:.1f}% ({agent_idx + 1}/{total_agents} agents)")
        if eta > 0:
            print(f"   Estimated remaining time: {eta:.0f}s")

        # INCREMENTAL STORAGE: All 10 agents
        if (agent_idx + 1) % save_interval == 0 or (agent_idx + 1) == total_agents:
            print(f"\n💾 Save after every {agent_idx + 1} Agent...")

            # Create DataFrames
            df_survey_results = pd.DataFrame(all_survey_results)
            if len(df_survey_results) > 0:
                df_pivot = df_survey_results.pivot_table(
                    index=['username', 'brand'],
                    columns='question',
                    values='rating'
                ).reset_index()
            else:
                df_pivot = pd.DataFrame()

            # Pre-Learning Results DataFrame
            df_pre_learning_results = pd.DataFrame(all_pre_learning_results)
            if len(df_pre_learning_results) > 0:
                df_pre_learning_pivot = df_pre_learning_results.pivot_table(
                    index=['username', 'brand'],
                    columns='question',
                    values='rating'
                ).reset_index()
            else:
                df_pre_learning_pivot = pd.DataFrame()

            # Brand Analysis Tracking DataFrame
            df_brand_analysis_tracking = pd.DataFrame(all_brand_analysis_tracking)

            # Reasoning-data with Confidence
            df_reasoning = pd.DataFrame(all_reasoning_data)

            # Agent Communication Logs
            df_survey_messages = pd.DataFrame(all_survey_messages)
            df_bidding_messages = pd.DataFrame(all_bidding_messages)
            df_feedback_messages = pd.DataFrame(all_feedback_messages)

            # Bidding-results
            df_bidding = pd.DataFrame(all_bidding_results)

            # Learning Summary
            learning_summary = {
                "survey_adjustments": shared_learning_memory.learning_feedback["survey_adjustments"],
                "bidding_adjustments": shared_learning_memory.learning_feedback["bidding_adjustments"],
                "wisdom_count": len(shared_learning_memory.learning_feedback["accumulated_wisdom"]),
                "agents_processed": agent_idx + 1,
                "timestamp": datetime.now().isoformat()
            }

            # Save with checkpoint number
            checkpoint_num = (agent_idx + 1) // save_interval
            checkpoint_suffix = f"checkpoint{checkpoint_num}_{agent_idx + 1}agents"

            # Survey-files
            if len(df_pivot) > 0:
                df_pivot.to_csv(f"{output_dir_survey}agent_survey_results_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_pre_learning_pivot) > 0:
                # Save Pre-Learning Results as Excel
                df_pre_learning_pivot.to_excel(f"{output_dir_survey}agent_survey_results_pre_learning_{base_timestamp}_{checkpoint_suffix}.xlsx", index=False)

            # Save Brand Analysis Tracking as Excel
            if len(df_brand_analysis_tracking) > 0:
                df_brand_analysis_tracking.to_excel(f"{output_dir_survey}agent_brand_analysis_impact_{base_timestamp}_{checkpoint_suffix}.xlsx", index=False)

            if len(df_reasoning) > 0:
                df_reasoning.to_csv(f"{output_dir_survey}agent_survey_reasoning_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_survey_messages) > 0:
                df_survey_messages.to_csv(f"{output_dir_survey}agent_survey_communications_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            # Bidding-files
            if len(df_bidding) > 0:
                df_bidding.to_csv(f"{output_dir_bidding}agent_bidding_results_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_bidding_messages) > 0:
                df_bidding_messages.to_csv(f"{output_dir_bidding}agent_bidding_communications_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            # Learning-files
            if len(df_feedback_messages) > 0:
                df_feedback_messages.to_csv(f"{output_dir_learning}agent_feedback_messages_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            with open(f"{output_dir_learning}agent_learning_summary_{base_timestamp}_{checkpoint_suffix}.json", "w") as f:
                json.dump(learning_summary, f, indent=2)

            print(f"   ✅ Checkpoint {checkpoint_num} saved!")
            print(f"      Survey Results (with Learning): {len(df_pivot)} Entries")
            print(f"      Survey Results (Pre-Learning): {len(df_pre_learning_pivot)} Entries")
            print(f"      Brand Analysis Impact: {len(df_brand_analysis_tracking)} Entries")
            print(f"      Reasoning Data: {len(df_reasoning)} Entries")
            print(f"      Bidding Results: {len(df_bidding)} bids")
            print(f"      Feedback Messages: {len(df_feedback_messages)} Messages")

        # Short break between agents
        if agent_idx < total_agents - 1:
            print("\n   ⏳ Waiting 2 seconds...")
            await asyncio.sleep(2)

    # FINAL RESULTS
    print("\n\n📊 Create final result DataFrames...")

    # Create final DataFrames
    df_survey_results = pd.DataFrame(all_survey_results)
    df_pivot = df_survey_results.pivot_table(
        index=['username', 'brand'],
        columns='question',
        values='rating'
    ).reset_index()

    # Pre-Learning Results
    df_pre_learning_results = pd.DataFrame(all_pre_learning_results)
    df_pre_learning_pivot = df_pre_learning_results.pivot_table(
        index=['username', 'brand'],
        columns='question',
        values='rating'
    ).reset_index()

    # Brand Analysis Tracking
    df_brand_analysis_tracking = pd.DataFrame(all_brand_analysis_tracking)

    df_reasoning = pd.DataFrame(all_reasoning_data)
    df_survey_messages = pd.DataFrame(all_survey_messages)
    df_bidding_messages = pd.DataFrame(all_bidding_messages)
    df_feedback_messages = pd.DataFrame(all_feedback_messages)
    df_bidding = pd.DataFrame(all_bidding_results)

    # Final Learning Summary
    learning_summary = {
        "survey_adjustments": shared_learning_memory.learning_feedback["survey_adjustments"],
        "bidding_adjustments": shared_learning_memory.learning_feedback["bidding_adjustments"],
        "wisdom_count": len(shared_learning_memory.learning_feedback["accumulated_wisdom"]),
        "final_insights": shared_learning_memory.learning_feedback["accumulated_wisdom"][-5:]
    }

    # Save final results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Survey files
    df_pivot.to_csv(f"{output_dir_survey}agent_survey_results_{timestamp}_final.csv", index=False)
    df_pre_learning_pivot.to_excel(f"{output_dir_survey}agent_survey_results_pre_learning_{timestamp}_final.xlsx", index=False)

    # Brand Analysis Impact as a final Excel file
    if len(df_brand_analysis_tracking) > 0:
        df_brand_analysis_tracking.to_excel(f"{output_dir_survey}agent_brand_analysis_impact_{timestamp}_final.xlsx", index=False)

    df_reasoning.to_csv(f"{output_dir_survey}agent_survey_reasoning_{timestamp}_final.csv", index=False)
    df_survey_messages.to_csv(f"{output_dir_survey}agent_survey_communications_{timestamp}_final.csv", index=False)

    # Bidding-files
    df_bidding.to_csv(f"{output_dir_bidding}agent_bidding_results_{timestamp}_final.csv", index=False)
    df_bidding_messages.to_csv(f"{output_dir_bidding}agent_bidding_communications_{timestamp}_final.csv", index=False)

    # Learning-files
    df_feedback_messages.to_csv(f"{output_dir_learning}agent_feedback_messages_{timestamp}_final.csv", index=False)
    with open(f"{output_dir_learning}agent_learning_summary_{timestamp}_final.json", "w") as f:
        json.dump(learning_summary, f, indent=2)

    print(f"\n💾 Alle finalen Ergebnisse gespeichert!")
    print(f"   Survey Results (mit Learning): {len(df_pivot)} Einträge")
    print(f"   Survey Results (Pre-Learning): {len(df_pre_learning_pivot)} Einträge")
    print(f"   Brand Analysis Impact: {len(df_brand_analysis_tracking)} Einträge")
    print(f"   Reasoning Data: {len(df_reasoning)} Einträge")
    print(f"   Survey Messages: {len(df_survey_messages)} Nachrichten")
    print(f"   Bidding Results: {len(df_bidding)} Gebote")
    print(f"   Bidding Messages: {len(df_bidding_messages)} Nachrichten")
    print(f"   Feedback Messages: {len(df_feedback_messages)} Feedback-Nachrichten")
    print(f"   Learning Summary: {len(learning_summary['survey_adjustments'])} Survey-Muster, {len(learning_summary['bidding_adjustments'])} Bidding-Muster")
    print(f"   Gesamtzeit: {time.time() - start_time:.1f} Sekunden")

    # Show Brand Analysis Impact statistics
    if len(df_brand_analysis_tracking) > 0:
        print("\n📊 Brand Analysis Impact statistics:")
        total_challenges = df_brand_analysis_tracking['challenge_received'].sum()
        total_rating_changes = (df_brand_analysis_tracking['rating_change'] != 0).sum()
        avg_rating_change = df_brand_analysis_tracking['rating_change'].mean()

        print(f"   Challenges: {total_challenges}")
        print(f"   Rating-changes: {total_rating_changes}")
        print(f"   Average Rating-change: {avg_rating_change:.2f}")

        # Rating changes per brand
        print("\n   Rating changes per brand:")
        brand_changes = df_brand_analysis_tracking.groupby('brand')['rating_change'].agg(['count', 'mean', 'sum'])
        print(brand_changes)

    # Show statistics
    if len(df_bidding) > 0:
        print("\n📊 Bidding statistics:")
        print(f"   Average bid: €{df_bidding['bid'].mean():.2f}")
        print(f"   Bid range: €{df_bidding['bid'].min():.2f} - €{df_bidding['bid'].max():.2f}")

        print("\n📈 Average bids per brand:")
        brand_stats = df_bidding.groupby('brand')['bid'].agg(['mean', 'std', 'count'])
        print(brand_stats)

        # Correlation between attachment and bids
        print("\n🔗 Correlation Attachment → Bids:")
        # Calculate correlations from collected data
        correlations = {}
        for brand in brands:
            brand_bids = df_bidding[df_bidding['brand'] == brand]
            if len(brand_bids) > 0:
                # Collect attachment scores from survey data
                attachments = []
                bids = []

                for _, bid_row in brand_bids.iterrows():
                    username = bid_row['username']
                    # Get attachment data from Survey Results
                    user_survey = df_pivot[df_pivot['username'] == username]
                    if len(user_survey) > 0:
                        brand_survey = user_survey[user_survey['brand'] == brand]
                        if len(brand_survey) > 0:
                            # Calculate average attachment score
                            attachment_cols = [col for col in df_pivot.columns if col.startswith('Q11_')]
                            if attachment_cols:
                                attachment_values = brand_survey[attachment_cols].values[0]
                                # Filter NaN values
                                valid_values = [v for v in attachment_values if pd.notna(v)]
                                if valid_values:
                                    avg_attachment = (sum(valid_values) / len(valid_values) - 1) / 4  # Convert 1-5 to 0-1
                                    attachments.append(avg_attachment)
                                    bids.append(bid_row['bid'])

                if len(attachments) > 1:
                    correlation = pd.Series(attachments).corr(pd.Series(bids))
                    correlations[brand] = correlation
                    print(f"   {brand}: r={correlation:.3f}")

    # Show final Learning Insights
    print("\n🧠 FINAL LEARNING INSIGHTS:")
    print("="*70)

    print("\n📊 Survey Learning Patterns:")
    for q_type, adj in learning_summary["survey_adjustments"].items():
        if adj["samples"] > 0:
            print(f"   {q_type}: {adj['avg_diff']:+.2f} (based on {adj['samples']} Samples)")

    print("\n💰 Bidding Learning Patterns:")
    for brand, adj in learning_summary["bidding_adjustments"].items():
        if adj["samples"] > 0:
            print(f"   {brand}: €{adj['avg_diff']:+.2f} (based on {adj['samples']} Samples)")

    print("\n💡 Top Accumulated Wisdom:")
    for idx, wisdom in enumerate(learning_summary.get("final_insights", [])[-3:], 1):
        if isinstance(wisdom, dict) and "insight" in wisdom:
            insight = wisdom["insight"]
            if isinstance(insight, dict) and "general_tendency" in insight:
                print(f"   {idx}. {insight['general_tendency']}")

    print("\n🎉 AGENT STUDY DONE!")
    print("="*70)
    print("\n📂 Saved files:")
    print(f"   Final results:")
    print(f"   - Survey Results (with Learning): agent_survey_results_{timestamp}_final.csv")
    print(f"   - Survey Results (Pre-Learning): agent_survey_results_pre_learning_{timestamp}_final.xlsx")
    print(f"   - Brand Analysis Impact: agent_brand_analysis_impact_{timestamp}_final.xlsx")
    print(f"   - Survey Reasoning: agent_survey_reasoning_{timestamp}_final.csv")
    print(f"   - Survey Communications: agent_survey_communications_{timestamp}_final.csv")
    print(f"   - Bidding Results: agent_bidding_results_{timestamp}_final.csv")
    print(f"   - Bidding Communications: agent_bidding_communications_{timestamp}_final.csv")
    print(f"   - Feedback Messages: agent_feedback_messages_{timestamp}_final.csv")
    print(f"   - Learning Summary: agent_learning_summary_{timestamp}_final.json")
    print(f"\n   Plus {(total_agents // save_interval)} Checkpoint files for incremental analysis")

    return df_pivot, df_pre_learning_pivot, df_reasoning, df_bidding, learning_summary, df_brand_analysis_tracking


async def run_complete_agent_survey_for_single_agent(profile: dict, memory: EnhancedMemoryModule, shared_memory: EnhancedMemoryModule):
    """Conducts the complete survey for a single agent and all brands"""

    agent_name = profile["username"]
    survey_results = []
    pre_learning_results = []
    reasoning_data = []
    agent_messages = []
    brand_analysis_tracking = []

    # Dictionary for agent responses (for feedback)
    agent_responses = {}

    print(f"\n{'='*70}")
    print(f"👤 Agent: {agent_name}")
    print(f"   Actual: {' > '.join(profile['actual_traits'][:2])}...")
    print(f"   Ideal: {' > '.join(profile['ideal_traits'][:2])}...")
    print(f"{'='*70}")

    # Empty STM, CoT History and Messages before each new person
    memory.short_term_memory.clear()
    memory.cot_history.clear()
    memory.agent_messages.clear()
    memory._survey_ratings.clear()

    # Survey for every brand
    for brand in brands:
        print(f"\n   📋 {brand} Survey with agent:")

        try:
            # Updated Call with Brand Analysis Tracking
            results, pre_learning, brand_analysis = await run_agent_survey(profile, brand, memory, shared_memory)

            # Collect data for DataFrame
            for question_code in ["Q9_consistent", "Q9_mirror", "Q10_consistent", "Q10_mirror"] + [f"Q11_{item}" for item in attachment_items]:
                rating = results[question_code]
                pre_learning_rating = pre_learning[question_code]

                # Save for feedback
                agent_responses[question_code] = rating

                survey_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Save Pre-Learning Results
                pre_learning_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": pre_learning_rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Find the corresponding reasoning from the memory
                reasoning_entry = next(
                    (m for m in memory.short_term_memory
                     if m.get("question", "").endswith(question_code) and m.get("brand") == brand),
                    None
                )

                if reasoning_entry:
                    reasoning_data.append({
                        "username": agent_name,
                        "brand": brand,
                        "question": question_code,
                        "rating": rating,
                        "pre_learning_rating": pre_learning_rating,
                        "reasoning": reasoning_entry["reasoning"],
                        "confidence": reasoning_entry.get("confidence", None)
                    })

            # Collect Brand Analysis Tracking
            brand_analysis_tracking.extend(brand_analysis)

            # Collect agent messages for this brand
            brand_messages = [msg for msg in memory.agent_messages
                            if any(brand in str(msg.content) for brand in [brand])]

            for msg in brand_messages:
                agent_messages.append({
                    "username": agent_name,
                    "brand": brand,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

        except Exception as e:
            print(f"   ❌ Error {brand}: {str(e)}")

    # Return Brand Analysis Tracking with
    return survey_results, pre_learning_results, reasoning_data, agent_messages, agent_responses, brand_analysis_tracking

print("✅ Sequential Agent Study runner defined!")
print("\n" + "="*70)
print("🚀 Ready to start")
print("="*70)
print("\nThe agent system processes each agent sequentially:")
print("   1️⃣ Create profile & memory → 2️⃣ Survey → 3️⃣ Bidding → 4️⃣ Feedback → 5️⃣ Learning Update")
print("\n💾 Automatic saving every 10 agents")
print("📊 Brand Analysis Impact Tracking as Excel-Export")

# Start
survey_results, pre_learning_results, reasoning_results, bidding_results, learning_summary, brand_analysis_tracking = await run_complete_agent_experiment()

In [ ]:
### The following code parts are necessary to continue the simulation in branches after it has been interrupted ###

In [ ]:
# =============================================================================
# CELL A: Setup and Profile-Loading in branches
# =============================================================================

import os, re, json, nest_asyncio, pandas as pd
from openai import OpenAI
from google.colab import drive
start_idx = 180  # Enter the first profile here that has not yet been translated, e.g., 180 here

nest_asyncio.apply()
drive.mount('/content/drive')

# UniGPT-Client
os.environ["OPENAI_API_KEY"]  = "sk-D7C8BrpeoPqd7qocibeI5Q"
os.environ["OPENAI_API_BASE"] = "https://gpt.uni-muenster.de/v1"

client = OpenAI(
    api_key  = os.environ["OPENAI_API_KEY"],
    base_url = os.environ["OPENAI_API_BASE"]
)
model = "Llama-3.3-70B"
print(f"✅ LLM Client configured: {model}")

# Import profile data from Excel
print("\n📊 Load profiles from Excel...")
df_profiles = pd.read_excel('/content/drive/MyDrive/profiles_survey/All_profiles.xlsx')
print(f"   Found: {len(df_profiles)} Profile")

# Helper function for filtering empty traits
def filter_traits(row, trait_type, max_count=4):
    """Filters empty/NaN traits from a row"""
    traits = []
    for i in range(1, max_count + 1):
        trait_value = row[f"{trait_type}_{i}"]
        # Check for NaN, None, empty strings or whitespace only
        if pd.notna(trait_value) and str(trait_value).strip():
            traits.append(str(trait_value).strip())
    return traits

# Convert profiles to structured format
profiles = []
for _, row in df_profiles.iterrows():
    actual_traits = filter_traits(row, "actual")
    ideal_traits = filter_traits(row, "ideal")

    profiles.append({
        "age":                  int(row.age),
        "income_range":         row.income,
        "occupation":           row.occupation,
        "gender":               row.gender,
        "actual_traits":        actual_traits,
        "actual_top_strength":  row.actual_top_strength,
        "ideal_traits":         ideal_traits,
        "ideal_top_strength":   row.ideal_top_strength,
        "username":             f"{row.username} Agent"
    })

# Translation function
def translate_to_english(text: str) -> str:
    """Translates text into English (if necessary)"""
    res = client.chat.completions.create(
        model   = model,
        messages= [
            {"role":"system","content":"You are a precise translator. If the input is already in English, return it exactly as-is. Otherwise translate literally without introducing synonyms or rephrasings."},
            {"role":"user","content":f"Translate into English (or return unchanged): \"{text}\""}
        ],
        temperature=0.1
    )
    return res.choices[0].message.content.strip()

# Pre-processing function for profiles
def preprocess_profile(profile: dict) -> dict:
    """Translates all traits of a profile into English"""
    p = dict(profile)
    p["actual_traits"]       = [translate_to_english(t) for t in p["actual_traits"]]
    p["actual_top_strength"] = translate_to_english(p["actual_top_strength"])
    p["ideal_traits"]        = [translate_to_english(t) for t in p["ideal_traits"]]
    p["ideal_top_strength"]  = translate_to_english(p["ideal_top_strength"])
    return p

# Only translate the profiles from index x onwards
print(f"\n🌐 Translate profiles from index {start_idx} into English...")
profiles_en = []                  # Leere Liste (falls neu gestartet)

for i, p in enumerate(profiles[start_idx:], start=start_idx):
    print(f"   Process {i+1}/{len(profiles)}: {p['username']}...", end=" ")
    profiles_en.append(preprocess_profile(p))
    print("✓")

print(f"\n✅ Total translated so far {len(profiles_en)} Profils (after Index {start_idx})")


# Reflection functions
def reflect_profile(profile: dict) -> str:
    """Creates a compact psychological summary of the profiles"""
    system_msg = (
        "You are a psychologist. Create a SINGLE PARAGRAPH psychological summary "
        "that integrates the person's actual self, ideal self, actual top strength, ideal top strength and demographics "
        "into a cohesive personality profile. Focus on the key psychological dynamics "
        "and self-concept. Keep it concise but insightful."
    )

    user_content = (
        f"Profile:\n"
        f"- Age: {profile['age']}, Gender: {profile['gender']}\n"
        f"- Occupation: {profile['occupation']}, Income: {profile['income_range']}\n"
        f"- Actual Self: {', '.join(profile['actual_traits'])} (top: {profile['actual_top_strength']})\n"
        f"- Ideal Self: {', '.join(profile['ideal_traits'])} (top: {profile['ideal_top_strength']})\n"
    )

    resp = client.chat.completions.create(
        model    = model,
        messages = [
            {"role":"system", "content": system_msg},
            {"role":"user",   "content": user_content}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return resp.choices[0].message.content.strip()

def reflect_actual_self(profile: dict) -> str:
    """Creates a psychological expert reflection on the Actual Self"""
    system_msg = (
        "You are a psychology expert. Analyze how this person sees themselves (actual self) "
        "based on their traits and top strength. Consider that traits are listed in descending order of importance. "
        "Focus on their self-perception, self-concept, and how they view their current personality. "
        "Keep it focused and concise."
    )

    user_content = (
        f"Actual Self Analysis:\n"
        f"- Actual traits (in order of importance): {', '.join(profile['actual_traits'])}\n"
        f"- Actual top strength: {profile['actual_top_strength']}\n"
        f"How does this person see themselves? What is their self-concept?"
    )

    resp = client.chat.completions.create(
        model    = model,
        messages = [
            {"role":"system", "content": system_msg},
            {"role":"user",   "content": user_content}
        ],
        temperature=0.7,
        max_tokens=300
    )
    return resp.choices[0].message.content.strip()

def reflect_ideal_self(profile: dict) -> str:
    """Creates a psychological expert reflection on the Ideal Self"""
    system_msg = (
        "You are a psychology expert. Analyze this person's ideal self and aspirations "
        "based on their ideal traits and top ideal strength. Consider that traits are listed in descending order of importance. "
        "Focus on their aspirations, what they want to become, and gaps between actual and ideal self. "
        "Keep it focused and concise."
    )

    user_content = (
        f"Ideal Self Analysis:\n"
        f"- Ideal traits (in order of importance): {', '.join(profile['ideal_traits'])}\n"
        f"- Ideal top strength: {profile['ideal_top_strength']}\n"
        f"What does this person aspire to be? What are their self-improvement goals?"
    )

    resp = client.chat.completions.create(
        model    = model,
        messages = [
            {"role":"system", "content": system_msg},
            {"role":"user",   "content": user_content}
        ],
        temperature=0.7,
        max_tokens=300
    )
    return resp.choices[0].message.content.strip()

# Initialize helper tool orchestrators (global, one-time)
print("\n🤖 Initialize Helper Tool Orchestrators...")
# Renaming
brand_personality_agent = BrandPersonalityToolOrchestrator()
price_research_agent = PriceResearchToolOrchestrator()
feedback_agent = FeedbackToolOrchestrator()

print("\n✅ Setup done!")
print(f"   Number Profiles: {len(profiles_en)}")
print("   Helper Tool Orchestrators: Ready")

brands = ["Nike", "Apple", "Levi's"]

In [ ]:
# =============================================================================
# CELL B: Access to shared memory
# =============================================================================

##the checkpoint files need to get pasted in here

import json
from datetime import datetime

# 1. Create a shared learning memory again
shared_learning_memory = EnhancedMemoryModule(
    agent_name="SharedLearning",
    max_short_term_size=100
)

# 2. List of the checkpoint files
paths = [
    "/content/drive/MyDrive/agent_learning_results/"
    "agent_learning_summary_20250628_193840_checkpoint4_40agents.json",
    "/content/drive/MyDrive/agent_learning_results/"
    "agent_learning_summary_20250629_090026_checkpoint3_30agents.json"
]

# 3. Auxiliary structures for merging
merged_survey  = {}
merged_bidding = {}
merged_wisdom  = []

for path in paths:
    with open(path, "r") as f:
        chk = json.load(f)
    #  Survey adjustments
    for qtype, adj in chk["survey_adjustments"].items():
        total = adj["avg_diff"] * adj["samples"]
        if qtype not in merged_survey:
            merged_survey[qtype] = {"total_diff": total, "samples": adj["samples"]}
        else:
            merged_survey[qtype]["total_diff"] += total
            merged_survey[qtype]["samples"]    += adj["samples"]
    # Merging Bidding adjustments
    for brand, adj in chk["bidding_adjustments"].items():
        total = adj["avg_diff"] * adj["samples"]
        if brand not in merged_bidding:
            merged_bidding[brand] = {"total_diff": total, "samples": adj["samples"]}
        else:
            merged_bidding[brand]["total_diff"] += total
            merged_bidding[brand]["samples"]    += adj["samples"]
    # Attach wisdom
    merged_wisdom.extend(chk.get("accumulated_wisdom", []))

# 4. Recalculate avg_diff
for d in merged_survey.values():
    d["avg_diff"] = d["total_diff"] / d["samples"]
for d in merged_bidding.values():
    d["avg_diff"] = d["total_diff"] / d["samples"]

# 5. Save to shared memory
shared_learning_memory.learning_feedback["survey_adjustments"]  = merged_survey
shared_learning_memory.learning_feedback["bidding_adjustments"] = merged_bidding
shared_learning_memory.learning_feedback["accumulated_wisdom"]  = merged_wisdom

print("✅ Merged Learning Memory with",
      f"{sum(d['samples'] for d in merged_survey.values())} Surveys,",
      f"{sum(d['samples'] for d in merged_bidding.values())} Bids,",
      f"{len(merged_wisdom)} Wisdom-Entrances")


In [ ]:
# =============================================================================
# CELL C: Starting the study again
# =============================================================================

async def run_complete_agent_experiment():
    """Runs the complete study"""

    print("🚀 Start study")
    print("="*70)

    # Collection storage for all results
    all_survey_results = []
    all_pre_learning_results = []
    all_reasoning_data = []
    all_survey_messages = []
    all_bidding_results = []
    all_bidding_messages = []
    all_feedback_messages = []
    all_brand_analysis_tracking = []

    total_agents = len(profiles_en)
    start_time = time.time()
    save_interval = 10  # Save all 10 agents

    # Output directories
    output_dir_survey = "/content/drive/MyDrive/agent_survey_results/"
    output_dir_bidding = "/content/drive/MyDrive/agent_bidding_results/"
    output_dir_learning = "/content/drive/MyDrive/agent_learning_results/"

    import os
    os.makedirs(output_dir_survey, exist_ok=True)
    os.makedirs(output_dir_bidding, exist_ok=True)
    os.makedirs(output_dir_learning, exist_ok=True)

    # Base timestamp for consistent file names
    base_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # MAIN LOOP:
    for agent_idx, profile in enumerate(profiles_en):

        agent_name = profile["username"]

        print(f"\n{'#'*80}")
        print(f"# AGENT {agent_idx + 1}/{total_agents}: {agent_name}")
        print(f"{'#'*80}")

        # STEP 1: Profile processing and reflections
        print(f"\n🔮 PHASE 1: PROFILE PROCESSING for {agent_name}")
        print("-" * 50)

        # Create memory module for this agent
        print(f"   🧠 Create memory module for {agent_name}...")
        memory = EnhancedMemoryModule(
            agent_name=agent_name,
            max_short_term_size=20
        )

        # Use global embedding model for efficiency
        if 'embeddings_model' not in globals():
            print("   📥 Load Embedding-Modell...")
            from langchain_community.embeddings import HuggingFaceEmbeddings
            embeddings_model = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
        memory.embeddings = embeddings_model

        # Create psychological reflections
        print("   🔮 Create psychological summaries...")
        general_summary = reflect_profile(profile)
        actual_self_summary = reflect_actual_self(profile)
        ideal_self_summary = reflect_ideal_self(profile)

        # Save profile and all reflections
        memory.store_profile(profile, general_summary, actual_self_summary, ideal_self_summary)

        print(f"   📝 General: {general_summary[:100]}...")
        print(f"   🎯 Actual Self: {actual_self_summary[:80]}...")
        print(f"   ⭐ Ideal Self: {ideal_self_summary[:80]}...")
        print(f"   ✅ Memory Module for {agent_name} created!")

        # Dictionary for all agent responses
        all_agent_responses = {}

        # STEP 2: Survey for this agent
        print(f"\n📋 PHASE 2: SURVEY for {agent_name}")
        print("-" * 50)

        # Updated Call with Brand Analysis Tracking
        agent_survey_results, agent_pre_learning_results, agent_reasoning_data, agent_survey_messages, agent_survey_responses, agent_brand_analysis_tracking = \
            await run_complete_agent_survey_for_single_agent(profile, memory, shared_learning_memory)

        # Collect Survey-results
        all_survey_results.extend(agent_survey_results)
        all_pre_learning_results.extend(agent_pre_learning_results)
        all_reasoning_data.extend(agent_reasoning_data)
        all_survey_messages.extend(agent_survey_messages)
        all_brand_analysis_tracking.extend(agent_brand_analysis_tracking)

        # Update agent responses dictionary
        all_agent_responses.update(agent_survey_responses)

        print(f"\n✅ Survey for {agent_name} done!")

        # Show Brand Attachment Scores
        print("   📊 Brand Attachment Scores:")
        for brand in brands:
            attachment = calculate_brand_attachment_score(memory, brand)
            print(f"      - {brand}: {attachment:.2%}")

        # STEP 3: Bidding for this agent
        print(f"\n🛒 PHASE 3: Bidding for {agent_name}")
        print("-" * 50)

        # Clear agent messages for bidding
        memory.agent_messages.clear()

        try:
            bidding_results = await run_agent_shop_bot(profile, memory, shared_learning_memory)
            all_bidding_results.extend(bidding_results)

            # Add bidding results to agent_responses
            for bid_result in bidding_results:
                brand = bid_result["brand"]
                all_agent_responses[f"bid_{brand}"] = bid_result["bid"]

            # Collect Agent Messages from bidding
            for msg in memory.agent_messages:
                all_bidding_messages.append({
                    "username": agent_name,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

            print(f"\n✅ Bidding for {agent_name} done!")

        except Exception as e:
            print(f"\n   ❌ Critical error when bidding for {agent_name}: {e!s}")

        # SCHRITT 4: FEEDBACK LOOP
        print(f"\n📊 PHASE 4: FEEDBACK for {agent_name}")
        print("-" * 50)

        # Generate feedback for each brand
        for brand in brands:
            feedback_msg = feedback_agent.create_feedback_message(all_agent_responses, profile, brand)

            # Save feedback in shared memory
            shared_learning_memory.add_learning_feedback(feedback_msg)

            # Also save in the individual memory
            memory.add_agent_message(feedback_msg)

            # Collect for logging
            all_feedback_messages.append({
                "username": agent_name,
                "brand": brand,
                "from_agent": feedback_msg.from_agent,
                "to_agent": feedback_msg.to_agent,
                "message_type": feedback_msg.message_type.value,
                "priority": feedback_msg.priority,
                "profile_match_score": feedback_msg.content.get("profile_match_score", 0),
                "matches_used": feedback_msg.content.get("matches_used", 0),
                "recommendations": json.dumps(feedback_msg.content.get("recommendations", {})),
                "timestamp": feedback_msg.timestamp
            })

            print(f"   📚 Feedback generated for {brand} and saved in the shared memory")

        # Show Learning Summary
        print(f"\n   📈 Learning Summary after {agent_name}:")
        print(f"      Survey Adjustments: {len(shared_learning_memory.learning_feedback['survey_adjustments'])} Pattern")
        print(f"      Bidding Adjustments: {len(shared_learning_memory.learning_feedback['bidding_adjustments'])} Pattern")
        print(f"      Accumulated Wisdom: {len(shared_learning_memory.learning_feedback['accumulated_wisdom'])} Insights")

        # Progress bar
        elapsed = time.time() - start_time
        progress = ((agent_idx + 1) / total_agents) * 100
        eta = (elapsed / (agent_idx + 1)) * (total_agents - agent_idx - 1) if agent_idx < total_agents - 1 else 0

        print(f"\n📈 Overall progress: {progress:.1f}% ({agent_idx + 1}/{total_agents} agents)")
        if eta > 0:
            print(f"   Estimated remaining time: {eta:.0f}s")

        # INCREMENTAL STORAGE: All 10 agents
        if (agent_idx + 1) % save_interval == 0 or (agent_idx + 1) == total_agents:
            print(f"\n💾 Save after every {agent_idx + 1} Agent...")

            # Create DataFrames
            df_survey_results = pd.DataFrame(all_survey_results)
            if len(df_survey_results) > 0:
                df_pivot = df_survey_results.pivot_table(
                    index=['username', 'brand'],
                    columns='question',
                    values='rating'
                ).reset_index()
            else:
                df_pivot = pd.DataFrame()

            # Pre-Learning Results DataFrame
            df_pre_learning_results = pd.DataFrame(all_pre_learning_results)
            if len(df_pre_learning_results) > 0:
                df_pre_learning_pivot = df_pre_learning_results.pivot_table(
                    index=['username', 'brand'],
                    columns='question',
                    values='rating'
                ).reset_index()
            else:
                df_pre_learning_pivot = pd.DataFrame()

            # Brand Analysis Tracking DataFrame
            df_brand_analysis_tracking = pd.DataFrame(all_brand_analysis_tracking)

            # Reasoning-data with Confidence
            df_reasoning = pd.DataFrame(all_reasoning_data)

            # Agent Communication Logs
            df_survey_messages = pd.DataFrame(all_survey_messages)
            df_bidding_messages = pd.DataFrame(all_bidding_messages)
            df_feedback_messages = pd.DataFrame(all_feedback_messages)

            # Bidding-results
            df_bidding = pd.DataFrame(all_bidding_results)

            # Learning Summary
            learning_summary = {
                "survey_adjustments": shared_learning_memory.learning_feedback["survey_adjustments"],
                "bidding_adjustments": shared_learning_memory.learning_feedback["bidding_adjustments"],
                "wisdom_count": len(shared_learning_memory.learning_feedback["accumulated_wisdom"]),
                "agents_processed": agent_idx + 1,
                "timestamp": datetime.now().isoformat()
            }

            # Save with checkpoint number
            checkpoint_num = (agent_idx + 1) // save_interval
            checkpoint_suffix = f"checkpoint{checkpoint_num}_{agent_idx + 1}agents"

            # Survey-files
            if len(df_pivot) > 0:
                df_pivot.to_csv(f"{output_dir_survey}agent_survey_results_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_pre_learning_pivot) > 0:
                # Save Pre-Learning Results as Excel
                df_pre_learning_pivot.to_excel(f"{output_dir_survey}agent_survey_results_pre_learning_{base_timestamp}_{checkpoint_suffix}.xlsx", index=False)

            # Save Brand Analysis Tracking as Excel
            if len(df_brand_analysis_tracking) > 0:
                df_brand_analysis_tracking.to_excel(f"{output_dir_survey}agent_brand_analysis_impact_{base_timestamp}_{checkpoint_suffix}.xlsx", index=False)

            if len(df_reasoning) > 0:
                df_reasoning.to_csv(f"{output_dir_survey}agent_survey_reasoning_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_survey_messages) > 0:
                df_survey_messages.to_csv(f"{output_dir_survey}agent_survey_communications_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            # Bidding-files
            if len(df_bidding) > 0:
                df_bidding.to_csv(f"{output_dir_bidding}agent_bidding_results_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_bidding_messages) > 0:
                df_bidding_messages.to_csv(f"{output_dir_bidding}agent_bidding_communications_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            # Learning-files
            if len(df_feedback_messages) > 0:
                df_feedback_messages.to_csv(f"{output_dir_learning}agent_feedback_messages_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            with open(f"{output_dir_learning}agent_learning_summary_{base_timestamp}_{checkpoint_suffix}.json", "w") as f:
                json.dump(learning_summary, f, indent=2)

            print(f"   ✅ Checkpoint {checkpoint_num} saved!")
            print(f"      Survey Results (with Learning): {len(df_pivot)} Entries")
            print(f"      Survey Results (Pre-Learning): {len(df_pre_learning_pivot)} Entries")
            print(f"      Brand Analysis Impact: {len(df_brand_analysis_tracking)} Entries")
            print(f"      Reasoning Data: {len(df_reasoning)} Entries")
            print(f"      Bidding Results: {len(df_bidding)} bids")
            print(f"      Feedback Messages: {len(df_feedback_messages)} Messages")

        # Short break between agents
        if agent_idx < total_agents - 1:
            print("\n   ⏳ Waiting 2 seconds...")
            await asyncio.sleep(2)

    # FINAL RESULTS
    print("\n\n📊 Create final result DataFrames...")

    # Create final DataFrames
    df_survey_results = pd.DataFrame(all_survey_results)
    df_pivot = df_survey_results.pivot_table(
        index=['username', 'brand'],
        columns='question',
        values='rating'
    ).reset_index()

    # Pre-Learning Results
    df_pre_learning_results = pd.DataFrame(all_pre_learning_results)
    df_pre_learning_pivot = df_pre_learning_results.pivot_table(
        index=['username', 'brand'],
        columns='question',
        values='rating'
    ).reset_index()

    # Brand Analysis Tracking
    df_brand_analysis_tracking = pd.DataFrame(all_brand_analysis_tracking)

    df_reasoning = pd.DataFrame(all_reasoning_data)
    df_survey_messages = pd.DataFrame(all_survey_messages)
    df_bidding_messages = pd.DataFrame(all_bidding_messages)
    df_feedback_messages = pd.DataFrame(all_feedback_messages)
    df_bidding = pd.DataFrame(all_bidding_results)

    # Final Learning Summary
    learning_summary = {
        "survey_adjustments": shared_learning_memory.learning_feedback["survey_adjustments"],
        "bidding_adjustments": shared_learning_memory.learning_feedback["bidding_adjustments"],
        "wisdom_count": len(shared_learning_memory.learning_feedback["accumulated_wisdom"]),
        "final_insights": shared_learning_memory.learning_feedback["accumulated_wisdom"][-5:]
    }

    # Save final results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Survey files
    df_pivot.to_csv(f"{output_dir_survey}agent_survey_results_{timestamp}_final.csv", index=False)
    df_pre_learning_pivot.to_excel(f"{output_dir_survey}agent_survey_results_pre_learning_{timestamp}_final.xlsx", index=False)

    # Brand Analysis Impact as a final Excel file
    if len(df_brand_analysis_tracking) > 0:
        df_brand_analysis_tracking.to_excel(f"{output_dir_survey}agent_brand_analysis_impact_{timestamp}_final.xlsx", index=False)

    df_reasoning.to_csv(f"{output_dir_survey}agent_survey_reasoning_{timestamp}_final.csv", index=False)
    df_survey_messages.to_csv(f"{output_dir_survey}agent_survey_communications_{timestamp}_final.csv", index=False)

    # Bidding-files
    df_bidding.to_csv(f"{output_dir_bidding}agent_bidding_results_{timestamp}_final.csv", index=False)
    df_bidding_messages.to_csv(f"{output_dir_bidding}agent_bidding_communications_{timestamp}_final.csv", index=False)

    # Learning-files
    df_feedback_messages.to_csv(f"{output_dir_learning}agent_feedback_messages_{timestamp}_final.csv", index=False)
    with open(f"{output_dir_learning}agent_learning_summary_{timestamp}_final.json", "w") as f:
        json.dump(learning_summary, f, indent=2)

    print(f"\n💾 Alle finalen Ergebnisse gespeichert!")
    print(f"   Survey Results (mit Learning): {len(df_pivot)} Einträge")
    print(f"   Survey Results (Pre-Learning): {len(df_pre_learning_pivot)} Einträge")
    print(f"   Brand Analysis Impact: {len(df_brand_analysis_tracking)} Einträge")
    print(f"   Reasoning Data: {len(df_reasoning)} Einträge")
    print(f"   Survey Messages: {len(df_survey_messages)} Nachrichten")
    print(f"   Bidding Results: {len(df_bidding)} Gebote")
    print(f"   Bidding Messages: {len(df_bidding_messages)} Nachrichten")
    print(f"   Feedback Messages: {len(df_feedback_messages)} Feedback-Nachrichten")
    print(f"   Learning Summary: {len(learning_summary['survey_adjustments'])} Survey-Muster, {len(learning_summary['bidding_adjustments'])} Bidding-Muster")
    print(f"   Gesamtzeit: {time.time() - start_time:.1f} Sekunden")

    # Show Brand Analysis Impact statistics
    if len(df_brand_analysis_tracking) > 0:
        print("\n📊 Brand Analysis Impact statistics:")
        total_challenges = df_brand_analysis_tracking['challenge_received'].sum()
        total_rating_changes = (df_brand_analysis_tracking['rating_change'] != 0).sum()
        avg_rating_change = df_brand_analysis_tracking['rating_change'].mean()

        print(f"   Challenges: {total_challenges}")
        print(f"   Rating-changes: {total_rating_changes}")
        print(f"   Average Rating-change: {avg_rating_change:.2f}")

        # Rating changes per brand
        print("\n   Rating changes per brand:")
        brand_changes = df_brand_analysis_tracking.groupby('brand')['rating_change'].agg(['count', 'mean', 'sum'])
        print(brand_changes)

    # Show statistics
    if len(df_bidding) > 0:
        print("\n📊 Bidding statistics:")
        print(f"   Average bid: €{df_bidding['bid'].mean():.2f}")
        print(f"   Bid range: €{df_bidding['bid'].min():.2f} - €{df_bidding['bid'].max():.2f}")

        print("\n📈 Average bids per brand:")
        brand_stats = df_bidding.groupby('brand')['bid'].agg(['mean', 'std', 'count'])
        print(brand_stats)

        # Correlation between attachment and bids
        print("\n🔗 Correlation Attachment → Bids:")
        # Calculate correlations from collected data
        correlations = {}
        for brand in brands:
            brand_bids = df_bidding[df_bidding['brand'] == brand]
            if len(brand_bids) > 0:
                # Collect attachment scores from survey data
                attachments = []
                bids = []

                for _, bid_row in brand_bids.iterrows():
                    username = bid_row['username']
                    # Get attachment data from Survey Results
                    user_survey = df_pivot[df_pivot['username'] == username]
                    if len(user_survey) > 0:
                        brand_survey = user_survey[user_survey['brand'] == brand]
                        if len(brand_survey) > 0:
                            # Calculate average attachment score
                            attachment_cols = [col for col in df_pivot.columns if col.startswith('Q11_')]
                            if attachment_cols:
                                attachment_values = brand_survey[attachment_cols].values[0]
                                # Filter NaN values
                                valid_values = [v for v in attachment_values if pd.notna(v)]
                                if valid_values:
                                    avg_attachment = (sum(valid_values) / len(valid_values) - 1) / 4  # Convert 1-5 to 0-1
                                    attachments.append(avg_attachment)
                                    bids.append(bid_row['bid'])

                if len(attachments) > 1:
                    correlation = pd.Series(attachments).corr(pd.Series(bids))
                    correlations[brand] = correlation
                    print(f"   {brand}: r={correlation:.3f}")

    # Show final Learning Insights
    print("\n🧠 FINAL LEARNING INSIGHTS:")
    print("="*70)

    print("\n📊 Survey Learning Patterns:")
    for q_type, adj in learning_summary["survey_adjustments"].items():
        if adj["samples"] > 0:
            print(f"   {q_type}: {adj['avg_diff']:+.2f} (based on {adj['samples']} Samples)")

    print("\n💰 Bidding Learning Patterns:")
    for brand, adj in learning_summary["bidding_adjustments"].items():
        if adj["samples"] > 0:
            print(f"   {brand}: €{adj['avg_diff']:+.2f} (based on {adj['samples']} Samples)")

    print("\n💡 Top Accumulated Wisdom:")
    for idx, wisdom in enumerate(learning_summary.get("final_insights", [])[-3:], 1):
        if isinstance(wisdom, dict) and "insight" in wisdom:
            insight = wisdom["insight"]
            if isinstance(insight, dict) and "general_tendency" in insight:
                print(f"   {idx}. {insight['general_tendency']}")

    print("\n🎉 AGENT STUDY DONE!")
    print("="*70)
    print("\n📂 Saved files:")
    print(f"   Final results:")
    print(f"   - Survey Results (with Learning): agent_survey_results_{timestamp}_final.csv")
    print(f"   - Survey Results (Pre-Learning): agent_survey_results_pre_learning_{timestamp}_final.xlsx")
    print(f"   - Brand Analysis Impact: agent_brand_analysis_impact_{timestamp}_final.xlsx")
    print(f"   - Survey Reasoning: agent_survey_reasoning_{timestamp}_final.csv")
    print(f"   - Survey Communications: agent_survey_communications_{timestamp}_final.csv")
    print(f"   - Bidding Results: agent_bidding_results_{timestamp}_final.csv")
    print(f"   - Bidding Communications: agent_bidding_communications_{timestamp}_final.csv")
    print(f"   - Feedback Messages: agent_feedback_messages_{timestamp}_final.csv")
    print(f"   - Learning Summary: agent_learning_summary_{timestamp}_final.json")
    print(f"\n   Plus {(total_agents // save_interval)} Checkpoint files for incremental analysis")

    return df_pivot, df_pre_learning_pivot, df_reasoning, df_bidding, learning_summary, df_brand_analysis_tracking


async def run_complete_agent_survey_for_single_agent(profile: dict, memory: EnhancedMemoryModule, shared_memory: EnhancedMemoryModule):
    """Conducts the complete survey for a single agent and all brands"""

    agent_name = profile["username"]
    survey_results = []
    pre_learning_results = []
    reasoning_data = []
    agent_messages = []
    brand_analysis_tracking = []

    # Dictionary for agent responses (for feedback)
    agent_responses = {}

    print(f"\n{'='*70}")
    print(f"👤 Agent: {agent_name}")
    print(f"   Actual: {' > '.join(profile['actual_traits'][:2])}...")
    print(f"   Ideal: {' > '.join(profile['ideal_traits'][:2])}...")
    print(f"{'='*70}")

    # Empty STM, CoT History and Messages before each new person
    memory.short_term_memory.clear()
    memory.cot_history.clear()
    memory.agent_messages.clear()
    memory._survey_ratings.clear()

    # Survey for every brand
    for brand in brands:
        print(f"\n   📋 {brand} Survey with agent:")

        try:
            # Updated Call with Brand Analysis Tracking
            results, pre_learning, brand_analysis = await run_agent_survey(profile, brand, memory, shared_memory)

            # Collect data for DataFrame
            for question_code in ["Q9_consistent", "Q9_mirror", "Q10_consistent", "Q10_mirror"] + [f"Q11_{item}" for item in attachment_items]:
                rating = results[question_code]
                pre_learning_rating = pre_learning[question_code]

                # Save for feedback
                agent_responses[question_code] = rating

                survey_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Save Pre-Learning Results
                pre_learning_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": pre_learning_rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Find the corresponding reasoning from the memory
                reasoning_entry = next(
                    (m for m in memory.short_term_memory
                     if m.get("question", "").endswith(question_code) and m.get("brand") == brand),
                    None
                )

                if reasoning_entry:
                    reasoning_data.append({
                        "username": agent_name,
                        "brand": brand,
                        "question": question_code,
                        "rating": rating,
                        "pre_learning_rating": pre_learning_rating,
                        "reasoning": reasoning_entry["reasoning"],
                        "confidence": reasoning_entry.get("confidence", None)
                    })

            # Collect Brand Analysis Tracking
            brand_analysis_tracking.extend(brand_analysis)

            # Collect agent messages for this brand
            brand_messages = [msg for msg in memory.agent_messages
                            if any(brand in str(msg.content) for brand in [brand])]

            for msg in brand_messages:
                agent_messages.append({
                    "username": agent_name,
                    "brand": brand,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

        except Exception as e:
            print(f"   ❌ Error {brand}: {str(e)}")

    # Return Brand Analysis Tracking with
    return survey_results, pre_learning_results, reasoning_data, agent_messages, agent_responses, brand_analysis_tracking

print("✅ Sequential Agent Study runner defined!")
print("\n" + "="*70)
print("🚀 Ready to start")
print("="*70)
print("\nThe agent system processes each agent sequentially:")
print("   1️⃣ Create profile & memory → 2️⃣ Survey → 3️⃣ Bidding → 4️⃣ Feedback → 5️⃣ Learning Update")
print("\n💾 Automatic saving every 10 agents")
print("📊 Brand Analysis Impact Tracking as Excel-Export")

# Start
survey_results, pre_learning_results, reasoning_results, bidding_results, learning_summary, brand_analysis_tracking = await run_complete_agent_experiment()

In [ ]:
# Next time its stops same procedure. Cell A need to be adapted first.

# =============================================================================
# CELL B.1
# =============================================================================

import json
from datetime import datetime

# ① Neues Shared Learning Memory anlegen
shared_learning_memory = EnhancedMemoryModule(
    agent_name="SharedLearning",
    max_short_term_size=500
)

# ② Liste aller drei Checkpoint-Dateien
paths = [
    "/content/drive/MyDrive/agent_learning_results/"
    "agent_learning_summary_20250628_193840_checkpoint4_40agents.json",
    "/content/drive/MyDrive/agent_learning_results/"
    "agent_learning_summary_20250629_090026_checkpoint3_30agents.json",
    "/content/drive/MyDrive/agent_learning_results/"
    "agent_learning_summary_20250629_213736_checkpoint3_30agents.json"
]

# ③ Hilfsstrukturen zum Zusammenführen
merged_survey  = {}
merged_bidding = {}
merged_wisdom  = []

for path in paths:
    with open(path, "r") as f:
        chk = json.load(f)
    # ► Survey adjustments mergen
    for qtype, adj in chk["survey_adjustments"].items():
        total = adj["avg_diff"] * adj["samples"]
        if qtype not in merged_survey:
            merged_survey[qtype] = {"total_diff": total, "samples": adj["samples"]}
        else:
            merged_survey[qtype]["total_diff"] += total
            merged_survey[qtype]["samples"]    += adj["samples"]
    # ► Bidding adjustments mergen
    for brand, adj in chk["bidding_adjustments"].items():
        total = adj["avg_diff"] * adj["samples"]
        if brand not in merged_bidding:
            merged_bidding[brand] = {"total_diff": total, "samples": adj["samples"]}
        else:
            merged_bidding[brand]["total_diff"] += total
            merged_bidding[brand]["samples"]    += adj["samples"]
    # ► Wisdom anhängen
    merged_wisdom.extend(chk.get("accumulated_wisdom", []))

# ④ avg_diff neu berechnen
for d in merged_survey.values():
    d["avg_diff"] = d["total_diff"] / d["samples"]
for d in merged_bidding.values():
    d["avg_diff"] = d["total_diff"] / d["samples"]

# ⑤ In Shared Memory speichern
shared_learning_memory.learning_feedback["survey_adjustments"]  = merged_survey
shared_learning_memory.learning_feedback["bidding_adjustments"] = merged_bidding
shared_learning_memory.learning_feedback["accumulated_wisdom"]  = merged_wisdom

print("✅ Merged Learning Memory mit",
      f"{sum(d['samples'] for d in merged_survey.values())} Umfragen,",
      f"{sum(d['samples'] for d in merged_bidding.values())} Bids,",
      f"{len(merged_wisdom)} Wisdom-Einträgen")


In [ ]:
# =============================================================================
# CELL C.1
# =============================================================================

async def run_complete_agent_experiment():
    """Runs the complete study"""

    print("🚀 Start study")
    print("="*70)


    # Collection storage for all results
    all_survey_results = []
    all_pre_learning_results = []
    all_reasoning_data = []
    all_survey_messages = []
    all_bidding_results = []
    all_bidding_messages = []
    all_feedback_messages = []
    all_brand_analysis_tracking = []

    total_agents = len(profiles_en)
    start_time = time.time()
    save_interval = 10  # Save all 10 agents

    # Output directories
    output_dir_survey = "/content/drive/MyDrive/agent_survey_results/"
    output_dir_bidding = "/content/drive/MyDrive/agent_bidding_results/"
    output_dir_learning = "/content/drive/MyDrive/agent_learning_results/"

    import os
    os.makedirs(output_dir_survey, exist_ok=True)
    os.makedirs(output_dir_bidding, exist_ok=True)
    os.makedirs(output_dir_learning, exist_ok=True)

    # Base timestamp for consistent file names
    base_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # MAIN LOOP:
    for agent_idx, profile in enumerate(profiles_en):

        agent_name = profile["username"]

        print(f"\n{'#'*80}")
        print(f"# AGENT {agent_idx + 1}/{total_agents}: {agent_name}")
        print(f"{'#'*80}")

        # STEP 1: Profile processing and reflections
        print(f"\n🔮 PHASE 1: PROFILE PROCESSING for {agent_name}")
        print("-" * 50)

        # Create memory module for this agent
        print(f"   🧠 Create memory module for {agent_name}...")
        memory = EnhancedMemoryModule(
            agent_name=agent_name,
            max_short_term_size=20
        )

        # Use global embedding model for efficiency
        if 'embeddings_model' not in globals():
            print("   📥 Load Embedding-Modell...")
            from langchain_community.embeddings import HuggingFaceEmbeddings
            embeddings_model = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
        memory.embeddings = embeddings_model

        # Create psychological reflections
        print("   🔮 Create psychological summaries...")
        general_summary = reflect_profile(profile)
        actual_self_summary = reflect_actual_self(profile)
        ideal_self_summary = reflect_ideal_self(profile)

        # Save profile and all reflections
        memory.store_profile(profile, general_summary, actual_self_summary, ideal_self_summary)

        print(f"   📝 General: {general_summary[:100]}...")
        print(f"   🎯 Actual Self: {actual_self_summary[:80]}...")
        print(f"   ⭐ Ideal Self: {ideal_self_summary[:80]}...")
        print(f"   ✅ Memory Module for {agent_name} created!")

        # Dictionary for all agent responses
        all_agent_responses = {}

        # STEP 2: Survey for this agent
        print(f"\n📋 PHASE 2: SURVEY for {agent_name}")
        print("-" * 50)

        # Updated Call with Brand Analysis Tracking
        agent_survey_results, agent_pre_learning_results, agent_reasoning_data, agent_survey_messages, agent_survey_responses, agent_brand_analysis_tracking = \
            await run_complete_agent_survey_for_single_agent(profile, memory, shared_learning_memory)

        # Collect Survey-results
        all_survey_results.extend(agent_survey_results)
        all_pre_learning_results.extend(agent_pre_learning_results)
        all_reasoning_data.extend(agent_reasoning_data)
        all_survey_messages.extend(agent_survey_messages)
        all_brand_analysis_tracking.extend(agent_brand_analysis_tracking)

        # Update agent responses dictionary
        all_agent_responses.update(agent_survey_responses)

        print(f"\n✅ Survey for {agent_name} done!")

        # Show Brand Attachment Scores
        print("   📊 Brand Attachment Scores:")
        for brand in brands:
            attachment = calculate_brand_attachment_score(memory, brand)
            print(f"      - {brand}: {attachment:.2%}")

        # STEP 3: Bidding for this agent
        print(f"\n🛒 PHASE 3: Bidding for {agent_name}")
        print("-" * 50)

        # Clear agent messages for bidding
        memory.agent_messages.clear()

        try:
            bidding_results = await run_agent_shop_bot(profile, memory, shared_learning_memory)
            all_bidding_results.extend(bidding_results)

            # Add bidding results to agent_responses
            for bid_result in bidding_results:
                brand = bid_result["brand"]
                all_agent_responses[f"bid_{brand}"] = bid_result["bid"]

            # Collect Agent Messages from bidding
            for msg in memory.agent_messages:
                all_bidding_messages.append({
                    "username": agent_name,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

            print(f"\n✅ Bidding for {agent_name} done!")

        except Exception as e:
            print(f"\n   ❌ Critical error when bidding for {agent_name}: {e!s}")

        # SCHRITT 4: FEEDBACK LOOP
        print(f"\n📊 PHASE 4: FEEDBACK for {agent_name}")
        print("-" * 50)

        # Generate feedback for each brand
        for brand in brands:
            feedback_msg = feedback_agent.create_feedback_message(all_agent_responses, profile, brand)

            # Save feedback in shared memory
            shared_learning_memory.add_learning_feedback(feedback_msg)

            # Also save in the individual memory
            memory.add_agent_message(feedback_msg)

            # Collect for logging
            all_feedback_messages.append({
                "username": agent_name,
                "brand": brand,
                "from_agent": feedback_msg.from_agent,
                "to_agent": feedback_msg.to_agent,
                "message_type": feedback_msg.message_type.value,
                "priority": feedback_msg.priority,
                "profile_match_score": feedback_msg.content.get("profile_match_score", 0),
                "matches_used": feedback_msg.content.get("matches_used", 0),
                "recommendations": json.dumps(feedback_msg.content.get("recommendations", {})),
                "timestamp": feedback_msg.timestamp
            })

            print(f"   📚 Feedback generated for {brand} and saved in the shared memory")

        # Show Learning Summary
        print(f"\n   📈 Learning Summary after {agent_name}:")
        print(f"      Survey Adjustments: {len(shared_learning_memory.learning_feedback['survey_adjustments'])} Pattern")
        print(f"      Bidding Adjustments: {len(shared_learning_memory.learning_feedback['bidding_adjustments'])} Pattern")
        print(f"      Accumulated Wisdom: {len(shared_learning_memory.learning_feedback['accumulated_wisdom'])} Insights")

        # Progress bar
        elapsed = time.time() - start_time
        progress = ((agent_idx + 1) / total_agents) * 100
        eta = (elapsed / (agent_idx + 1)) * (total_agents - agent_idx - 1) if agent_idx < total_agents - 1 else 0

        print(f"\n📈 Overall progress: {progress:.1f}% ({agent_idx + 1}/{total_agents} agents)")
        if eta > 0:
            print(f"   Estimated remaining time: {eta:.0f}s")

        # INCREMENTAL STORAGE: All 10 agents
        if (agent_idx + 1) % save_interval == 0 or (agent_idx + 1) == total_agents:
            print(f"\n💾 Save after every {agent_idx + 1} Agent...")

            # Create DataFrames
            df_survey_results = pd.DataFrame(all_survey_results)
            if len(df_survey_results) > 0:
                df_pivot = df_survey_results.pivot_table(
                    index=['username', 'brand'],
                    columns='question',
                    values='rating'
                ).reset_index()
            else:
                df_pivot = pd.DataFrame()

            # Pre-Learning Results DataFrame
            df_pre_learning_results = pd.DataFrame(all_pre_learning_results)
            if len(df_pre_learning_results) > 0:
                df_pre_learning_pivot = df_pre_learning_results.pivot_table(
                    index=['username', 'brand'],
                    columns='question',
                    values='rating'
                ).reset_index()
            else:
                df_pre_learning_pivot = pd.DataFrame()

            # Brand Analysis Tracking DataFrame
            df_brand_analysis_tracking = pd.DataFrame(all_brand_analysis_tracking)

            # Reasoning-data with Confidence
            df_reasoning = pd.DataFrame(all_reasoning_data)

            # Agent Communication Logs
            df_survey_messages = pd.DataFrame(all_survey_messages)
            df_bidding_messages = pd.DataFrame(all_bidding_messages)
            df_feedback_messages = pd.DataFrame(all_feedback_messages)

            # Bidding-results
            df_bidding = pd.DataFrame(all_bidding_results)

            # Learning Summary
            learning_summary = {
                "survey_adjustments": shared_learning_memory.learning_feedback["survey_adjustments"],
                "bidding_adjustments": shared_learning_memory.learning_feedback["bidding_adjustments"],
                "wisdom_count": len(shared_learning_memory.learning_feedback["accumulated_wisdom"]),
                "agents_processed": agent_idx + 1,
                "timestamp": datetime.now().isoformat()
            }

            # Save with checkpoint number
            checkpoint_num = (agent_idx + 1) // save_interval
            checkpoint_suffix = f"checkpoint{checkpoint_num}_{agent_idx + 1}agents"

            # Survey-files
            if len(df_pivot) > 0:
                df_pivot.to_csv(f"{output_dir_survey}agent_survey_results_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_pre_learning_pivot) > 0:
                # Save Pre-Learning Results as Excel
                df_pre_learning_pivot.to_excel(f"{output_dir_survey}agent_survey_results_pre_learning_{base_timestamp}_{checkpoint_suffix}.xlsx", index=False)

            # Save Brand Analysis Tracking as Excel
            if len(df_brand_analysis_tracking) > 0:
                df_brand_analysis_tracking.to_excel(f"{output_dir_survey}agent_brand_analysis_impact_{base_timestamp}_{checkpoint_suffix}.xlsx", index=False)

            if len(df_reasoning) > 0:
                df_reasoning.to_csv(f"{output_dir_survey}agent_survey_reasoning_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_survey_messages) > 0:
                df_survey_messages.to_csv(f"{output_dir_survey}agent_survey_communications_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            # Bidding-files
            if len(df_bidding) > 0:
                df_bidding.to_csv(f"{output_dir_bidding}agent_bidding_results_{base_timestamp}_{checkpoint_suffix}.csv", index=False)
            if len(df_bidding_messages) > 0:
                df_bidding_messages.to_csv(f"{output_dir_bidding}agent_bidding_communications_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            # Learning-files
            if len(df_feedback_messages) > 0:
                df_feedback_messages.to_csv(f"{output_dir_learning}agent_feedback_messages_{base_timestamp}_{checkpoint_suffix}.csv", index=False)

            with open(f"{output_dir_learning}agent_learning_summary_{base_timestamp}_{checkpoint_suffix}.json", "w") as f:
                json.dump(learning_summary, f, indent=2)

            print(f"   ✅ Checkpoint {checkpoint_num} saved!")
            print(f"      Survey Results (with Learning): {len(df_pivot)} Entries")
            print(f"      Survey Results (Pre-Learning): {len(df_pre_learning_pivot)} Entries")
            print(f"      Brand Analysis Impact: {len(df_brand_analysis_tracking)} Entries")
            print(f"      Reasoning Data: {len(df_reasoning)} Entries")
            print(f"      Bidding Results: {len(df_bidding)} bids")
            print(f"      Feedback Messages: {len(df_feedback_messages)} Messages")

        # Short break between agents
        if agent_idx < total_agents - 1:
            print("\n   ⏳ Waiting 2 seconds...")
            await asyncio.sleep(2)

    # FINAL RESULTS
    print("\n\n📊 Create final result DataFrames...")

    # Create final DataFrames
    df_survey_results = pd.DataFrame(all_survey_results)
    df_pivot = df_survey_results.pivot_table(
        index=['username', 'brand'],
        columns='question',
        values='rating'
    ).reset_index()

    # Pre-Learning Results
    df_pre_learning_results = pd.DataFrame(all_pre_learning_results)
    df_pre_learning_pivot = df_pre_learning_results.pivot_table(
        index=['username', 'brand'],
        columns='question',
        values='rating'
    ).reset_index()

    # Brand Analysis Tracking
    df_brand_analysis_tracking = pd.DataFrame(all_brand_analysis_tracking)

    df_reasoning = pd.DataFrame(all_reasoning_data)
    df_survey_messages = pd.DataFrame(all_survey_messages)
    df_bidding_messages = pd.DataFrame(all_bidding_messages)
    df_feedback_messages = pd.DataFrame(all_feedback_messages)
    df_bidding = pd.DataFrame(all_bidding_results)

    # Final Learning Summary
    learning_summary = {
        "survey_adjustments": shared_learning_memory.learning_feedback["survey_adjustments"],
        "bidding_adjustments": shared_learning_memory.learning_feedback["bidding_adjustments"],
        "wisdom_count": len(shared_learning_memory.learning_feedback["accumulated_wisdom"]),
        "final_insights": shared_learning_memory.learning_feedback["accumulated_wisdom"][-5:]
    }

    # Save final results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Survey files
    df_pivot.to_csv(f"{output_dir_survey}agent_survey_results_{timestamp}_final.csv", index=False)
    df_pre_learning_pivot.to_excel(f"{output_dir_survey}agent_survey_results_pre_learning_{timestamp}_final.xlsx", index=False)

    # Brand Analysis Impact as a final Excel file
    if len(df_brand_analysis_tracking) > 0:
        df_brand_analysis_tracking.to_excel(f"{output_dir_survey}agent_brand_analysis_impact_{timestamp}_final.xlsx", index=False)

    df_reasoning.to_csv(f"{output_dir_survey}agent_survey_reasoning_{timestamp}_final.csv", index=False)
    df_survey_messages.to_csv(f"{output_dir_survey}agent_survey_communications_{timestamp}_final.csv", index=False)

    # Bidding-files
    df_bidding.to_csv(f"{output_dir_bidding}agent_bidding_results_{timestamp}_final.csv", index=False)
    df_bidding_messages.to_csv(f"{output_dir_bidding}agent_bidding_communications_{timestamp}_final.csv", index=False)

    # Learning-files
    df_feedback_messages.to_csv(f"{output_dir_learning}agent_feedback_messages_{timestamp}_final.csv", index=False)
    with open(f"{output_dir_learning}agent_learning_summary_{timestamp}_final.json", "w") as f:
        json.dump(learning_summary, f, indent=2)

    print(f"\n💾 Alle finalen Ergebnisse gespeichert!")
    print(f"   Survey Results (mit Learning): {len(df_pivot)} Einträge")
    print(f"   Survey Results (Pre-Learning): {len(df_pre_learning_pivot)} Einträge")
    print(f"   Brand Analysis Impact: {len(df_brand_analysis_tracking)} Einträge")
    print(f"   Reasoning Data: {len(df_reasoning)} Einträge")
    print(f"   Survey Messages: {len(df_survey_messages)} Nachrichten")
    print(f"   Bidding Results: {len(df_bidding)} Gebote")
    print(f"   Bidding Messages: {len(df_bidding_messages)} Nachrichten")
    print(f"   Feedback Messages: {len(df_feedback_messages)} Feedback-Nachrichten")
    print(f"   Learning Summary: {len(learning_summary['survey_adjustments'])} Survey-Muster, {len(learning_summary['bidding_adjustments'])} Bidding-Muster")
    print(f"   Gesamtzeit: {time.time() - start_time:.1f} Sekunden")

    # Show Brand Analysis Impact statistics
    if len(df_brand_analysis_tracking) > 0:
        print("\n📊 Brand Analysis Impact statistics:")
        total_challenges = df_brand_analysis_tracking['challenge_received'].sum()
        total_rating_changes = (df_brand_analysis_tracking['rating_change'] != 0).sum()
        avg_rating_change = df_brand_analysis_tracking['rating_change'].mean()

        print(f"   Challenges: {total_challenges}")
        print(f"   Rating-changes: {total_rating_changes}")
        print(f"   Average Rating-change: {avg_rating_change:.2f}")

        # Rating changes per brand
        print("\n   Rating changes per brand:")
        brand_changes = df_brand_analysis_tracking.groupby('brand')['rating_change'].agg(['count', 'mean', 'sum'])
        print(brand_changes)

    # Show statistics
    if len(df_bidding) > 0:
        print("\n📊 Bidding statistics:")
        print(f"   Average bid: €{df_bidding['bid'].mean():.2f}")
        print(f"   Bid range: €{df_bidding['bid'].min():.2f} - €{df_bidding['bid'].max():.2f}")

        print("\n📈 Average bids per brand:")
        brand_stats = df_bidding.groupby('brand')['bid'].agg(['mean', 'std', 'count'])
        print(brand_stats)

        # Correlation between attachment and bids
        print("\n🔗 Correlation Attachment → Bids:")
        # Calculate correlations from collected data
        correlations = {}
        for brand in brands:
            brand_bids = df_bidding[df_bidding['brand'] == brand]
            if len(brand_bids) > 0:
                # Collect attachment scores from survey data
                attachments = []
                bids = []

                for _, bid_row in brand_bids.iterrows():
                    username = bid_row['username']
                    # Get attachment data from Survey Results
                    user_survey = df_pivot[df_pivot['username'] == username]
                    if len(user_survey) > 0:
                        brand_survey = user_survey[user_survey['brand'] == brand]
                        if len(brand_survey) > 0:
                            # Calculate average attachment score
                            attachment_cols = [col for col in df_pivot.columns if col.startswith('Q11_')]
                            if attachment_cols:
                                attachment_values = brand_survey[attachment_cols].values[0]
                                # Filter NaN values
                                valid_values = [v for v in attachment_values if pd.notna(v)]
                                if valid_values:
                                    avg_attachment = (sum(valid_values) / len(valid_values) - 1) / 4  # Convert 1-5 to 0-1
                                    attachments.append(avg_attachment)
                                    bids.append(bid_row['bid'])

                if len(attachments) > 1:
                    correlation = pd.Series(attachments).corr(pd.Series(bids))
                    correlations[brand] = correlation
                    print(f"   {brand}: r={correlation:.3f}")

    # Show final Learning Insights
    print("\n🧠 FINAL LEARNING INSIGHTS:")
    print("="*70)

    print("\n📊 Survey Learning Patterns:")
    for q_type, adj in learning_summary["survey_adjustments"].items():
        if adj["samples"] > 0:
            print(f"   {q_type}: {adj['avg_diff']:+.2f} (based on {adj['samples']} Samples)")

    print("\n💰 Bidding Learning Patterns:")
    for brand, adj in learning_summary["bidding_adjustments"].items():
        if adj["samples"] > 0:
            print(f"   {brand}: €{adj['avg_diff']:+.2f} (based on {adj['samples']} Samples)")

    print("\n💡 Top Accumulated Wisdom:")
    for idx, wisdom in enumerate(learning_summary.get("final_insights", [])[-3:], 1):
        if isinstance(wisdom, dict) and "insight" in wisdom:
            insight = wisdom["insight"]
            if isinstance(insight, dict) and "general_tendency" in insight:
                print(f"   {idx}. {insight['general_tendency']}")

    print("\n🎉 AGENT STUDY DONE!")
    print("="*70)
    print("\n📂 Saved files:")
    print(f"   Final results:")
    print(f"   - Survey Results (with Learning): agent_survey_results_{timestamp}_final.csv")
    print(f"   - Survey Results (Pre-Learning): agent_survey_results_pre_learning_{timestamp}_final.xlsx")
    print(f"   - Brand Analysis Impact: agent_brand_analysis_impact_{timestamp}_final.xlsx")
    print(f"   - Survey Reasoning: agent_survey_reasoning_{timestamp}_final.csv")
    print(f"   - Survey Communications: agent_survey_communications_{timestamp}_final.csv")
    print(f"   - Bidding Results: agent_bidding_results_{timestamp}_final.csv")
    print(f"   - Bidding Communications: agent_bidding_communications_{timestamp}_final.csv")
    print(f"   - Feedback Messages: agent_feedback_messages_{timestamp}_final.csv")
    print(f"   - Learning Summary: agent_learning_summary_{timestamp}_final.json")
    print(f"\n   Plus {(total_agents // save_interval)} Checkpoint files for incremental analysis")

    return df_pivot, df_pre_learning_pivot, df_reasoning, df_bidding, learning_summary, df_brand_analysis_tracking


async def run_complete_agent_survey_for_single_agent(profile: dict, memory: EnhancedMemoryModule, shared_memory: EnhancedMemoryModule):
    """Conducts the complete survey for a single agent and all brands"""

    agent_name = profile["username"]
    survey_results = []
    pre_learning_results = []
    reasoning_data = []
    agent_messages = []
    brand_analysis_tracking = []

    # Dictionary for agent responses (for feedback)
    agent_responses = {}

    print(f"\n{'='*70}")
    print(f"👤 Agent: {agent_name}")
    print(f"   Actual: {' > '.join(profile['actual_traits'][:2])}...")
    print(f"   Ideal: {' > '.join(profile['ideal_traits'][:2])}...")
    print(f"{'='*70}")

    # Empty STM, CoT History and Messages before each new person
    memory.short_term_memory.clear()
    memory.cot_history.clear()
    memory.agent_messages.clear()
    memory._survey_ratings.clear()

    # Survey for every brand
    for brand in brands:
        print(f"\n   📋 {brand} Survey with agent:")

        try:
            # Updated Call with Brand Analysis Tracking
            results, pre_learning, brand_analysis = await run_agent_survey(profile, brand, memory, shared_memory)

            # Collect data for DataFrame
            for question_code in ["Q9_consistent", "Q9_mirror", "Q10_consistent", "Q10_mirror"] + [f"Q11_{item}" for item in attachment_items]:
                rating = results[question_code]
                pre_learning_rating = pre_learning[question_code]

                # Save for feedback
                agent_responses[question_code] = rating

                survey_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Save Pre-Learning Results
                pre_learning_results.append({
                    "username": agent_name,
                    "brand": brand,
                    "question": question_code,
                    "rating": pre_learning_rating,
                    "timestamp": datetime.now().isoformat()
                })

                # Find the corresponding reasoning from the memory
                reasoning_entry = next(
                    (m for m in memory.short_term_memory
                     if m.get("question", "").endswith(question_code) and m.get("brand") == brand),
                    None
                )

                if reasoning_entry:
                    reasoning_data.append({
                        "username": agent_name,
                        "brand": brand,
                        "question": question_code,
                        "rating": rating,
                        "pre_learning_rating": pre_learning_rating,
                        "reasoning": reasoning_entry["reasoning"],
                        "confidence": reasoning_entry.get("confidence", None)
                    })

            # Collect Brand Analysis Tracking
            brand_analysis_tracking.extend(brand_analysis)

            # Collect agent messages for this brand
            brand_messages = [msg for msg in memory.agent_messages
                            if any(brand in str(msg.content) for brand in [brand])]

            for msg in brand_messages:
                agent_messages.append({
                    "username": agent_name,
                    "brand": brand,
                    "from_agent": msg.from_agent,
                    "to_agent": msg.to_agent,
                    "message_type": msg.message_type.value,
                    "priority": msg.priority,
                    "content_summary": msg.content.get("summary", ""),
                    "timestamp": msg.timestamp
                })

        except Exception as e:
            print(f"   ❌ Error {brand}: {str(e)}")

    # Return Brand Analysis Tracking with
    return survey_results, pre_learning_results, reasoning_data, agent_messages, agent_responses, brand_analysis_tracking

print("✅ Sequential Agent Study runner defined!")
print("\n" + "="*70)
print("🚀 Ready to start")
print("="*70)
print("\nThe agent system processes each agent sequentially:")
print("   1️⃣ Create profile & memory → 2️⃣ Survey → 3️⃣ Bidding → 4️⃣ Feedback → 5️⃣ Learning Update")
print("\n💾 Automatic saving every 10 agents")
print("📊 Brand Analysis Impact Tracking as Excel-Export")

# Start
survey_results, pre_learning_results, reasoning_results, bidding_results, learning_summary, brand_analysis_tracking = await run_complete_agent_experiment()